### Imports & Properties

In [2]:
from helper_pkg import Helper
from helper_pkg.pyate import PYATE
from helper_pkg.cdax_utils import CDAX
hlp = Helper()

import pandas as pd
import numpy as np 
import re


from datetime import datetime, timedelta
import arrow

In [2]:
!nvidia-smi

Wed Mar  2 10:47:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA Tesla V1...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   35C    P0    37W / 300W |   2953MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Properties

rdr = 'RDR_MASTER'
notes = 'EWS_NOTES_MASTER'
notes1='EWS_NOTES_MASTER_dup'
geo_hier = 'QIES_Global_GeoHier'

id_col = 'case_id'
target_col = 'notes_history_cleaned_v3'
date_col = 'case_closed_date'

processed_table = f'EWS_PYATE_NOTES_PROCESSED'
processed_tbl=f'EWS_PYATE_NOTES_PROCESSED_dup'
restructured_table = f'EWS_PYATE_NOTES_RESTRUCTURED'
restructured_tbl = f'EWS_PYATE_NOTES_RESTRUCTURED_dup'
tfidf_table = f'{notes}_TFIDF'
tfidf_tbl = f'{notes1}_TFIDF'

dictionaries = {'ps': 'EWS_DICTIONARY_PS_UPDATED',
               'print': 'EWS_DICTIONARY_PRINT'}



current_date = arrow.utcnow()
start_date = (current_date - timedelta(days=365)).span('month')[0].format('YYYY-MM-DD')
start_dat = (current_date - timedelta(days=100)).span('month')[0].format('YYYY-MM-DD')

# start_date = '2020-04-01'
print('start_date:', start_date)
print('start_date1:', start_dat)
# start_date = '2021-05-01'

start_date: 2021-03-01
start_date1: 2021-11-01


In [4]:
def refresh_table_and_backups(df, table):
    drop_prev_2 = f"""drop table if exists pps_quality.{table};"""
    
    rename_prev_1_to_prev_2 = f"""alter table pps_quality.{table}_PREV_1
                                rename to {table}_PREV_2;"""
    
    rename_table_to_prev_1 = f"""alter table pps_quality.{table}
                                rename to {table}_PREV_1;"""
    
    hlp.run_query(drop_prev_2)
    hlp.run_query(rename_prev_1_to_prev_2)
    hlp.run_query(rename_table_to_prev_1)
    hlp.load_to_vertica_database(df, hlp.schema, table)
    

In [5]:
def adhoc_refresh(df, table):
    q = f"""drop table if exists pps_quality.{table};"""
    hlp.run_query(q)

    hlp.load_to_vertica_database(df, hlp.schema, table)

### IP2 Trend Detection - Shriya

In [7]:
import datetime as dt
query2 = """select max(case_closed_date)-12 as m from pps_quality.RDR_MASTER """
with hlp.get_connection('aws3') as aws_conn:
    dates = pd.read_sql(query2,aws_conn,parse_dates={'m':'%Y-%m-%d'})
cutoff_date = dates['m'][0].strftime('%Y-%m-%d')
print(cutoff_date)


2021-11-01


In [8]:
query = f"""Select ip2,
                    DATE_PART('Year', case_closed_date) || '-' || DATE_PART('Month', case_closed_date) || '-' || DATE_PART('day', case_closed_date) as Fiscal_Month, 
                    Count(*) 
                from pps_quality.RDR_MASTER
                where business = 'Printing'
                and segment = 'Commercial'
                and ip2_confidence>=0.5 
                and case_closed_date >= '{cutoff_date}'
                group by ip2,  DATE_PART('Year', case_closed_date) || '-' || DATE_PART('Month', case_closed_date) || '-' || DATE_PART('day', case_closed_date);"""

pd.set_option('display.max_rows',10000)
pd.set_option('display.max_columns',10000)

with hlp.get_connection('aws3') as aws_conn:
    df = pd.read_sql(query,aws_conn)

In [9]:
df['Fiscal_Month'] = pd.to_datetime(df['Fiscal_Month'], format='%Y-%m-%d', errors='ignore')

In [10]:
 df[['ip2', 'Fiscal_Month','Count']]

,ip2,Fiscal_Month,Count
0,adf,2021-11-07,38
1,hw_fax,2021-11-05,28
2,power_issue,2021-11-09,171
3,sw_installation,2021-11-08,253
4,print_mech_media_path,2021-11-07,81
5,noisy_product,2021-11-09,53
6,scan_copy_system,2021-11-11,171
7,display_front_panel,2021-11-10,104
8,sales,2021-11-11,33
9,hardware_ink_system,2021-11-10,54


In [11]:
# id_col = 'ip2'
# date_col = 'Fiscal_Month'
month_group = df[['ip2', 'Fiscal_Month','Count']].groupby(['ip2', pd.Grouper(key='Fiscal_Month', freq="1W")]).sum().unstack()
month_group.fillna(0, inplace=True)
month_group = month_group.droplevel([0], axis=1)

In [12]:
temp = month_group.copy()
temp.reset_index(inplace=True)

total_cases = len(df.ip2.unique())
# month_group = month_group.droplevel([0], axis=1)
month_cols = [col for col in month_group.columns if col != 'ip2']
# month_cols

In [13]:
col_rename_map = {}
for col in month_cols:
    col_rename_map.update({col: col.strftime('%y-%W')})

# col_rename_map

In [14]:
temp = temp.rename(col_rename_map, axis=1)
# temp

In [15]:
month_cols = temp.columns
for i in temp.columns:
    if 'Count' in i:
        print(i[1])
        month_cols.append(i[1])
# month_cols

In [20]:
curr = month_cols[(len(month_cols)-1)]
# print(month_cols[(len(month_cols)-1)])
prev = month_cols[(len(month_cols)-2)]
pre_prev = month_cols[(len(month_cols)-3)]

temp['curr_inc_flag'] = np.where((temp[curr] - temp[prev]) > 0, True, False)
temp['prev_inc_flag'] = np.where((temp[prev] - temp[pre_prev]) > 0, True, False)

temp['continuous_increase_flag'] = temp['curr_inc_flag'] & temp['prev_inc_flag']
temp['massive_increase'] = (((temp[curr] - temp[prev]) / temp[prev]) >= 0.25)

temp

In [63]:
ip2_list = temp.ip2[(temp.continuous_increase_flag==True) | (temp.massive_increase == True)].to_list()


len(ip2_list)
print(ip2_list)

['how_do_i_mac', 'print_mech_media_path', 'scan_copy_system', 'sw_installation']


### Refreshing EWS_NOTES_MASTER

In [6]:
# hlp.add_columns_to_table(notes, {'segment':'varchar(100)'})

In [7]:
# hlp.update_from_another_table(restructured_table, processed_table,
#                              'case_id',
#                              ['segment'], update_only_nulls=True)

In [6]:
query = f"""select rdr.case_id, 
                rdr.case_closed_date, 
                rdr.notes_history,
                rdr.notes_history_cleaned as notes_history_cleaned_v3,
                rdr.ip2, 
                rdr.platform_name, 
                rdr.business,
                rdr.country_code,
                rdr.segment
                --geo.Sub_Region1 as geo_market
            from {hlp.schema}.{rdr} rdr
            --left join {hlp.schema}.{geo_hier} geo
            --on rdr.country_code = geo.country_code 
            where date(case_closed_date) >= '{start_date}'
            and business is not null
            and platform_name is not null
            and notes_history is not null
            and ip2 is not null
            and case_id not in (select distinct case_id from {hlp.schema}.{notes})
            and language = 'en'
;"""

with hlp.get_connection() as conn:
    unprocessed = pd.read_sql(query, conn)
unprocessed

,case_id,case_closed_date,notes_history,notes_history_cleaned_v3,ip2,platform_name,business,country_code,segment
0,5080092962,2022-02-05 16:37:41,Log Type : Phone Log\r\nAction Type : Inbound ...,problem description: boot issue . computer boo...,boot_to_black_screen,PVAIO Sum13 Win8-64 NA Ambu,Personal Systems,US,Consumer
1,5080092989,2022-02-04 18:44:39,Log Type : Customer Log\r\nAction Type : Inbou...,scanner still wont work . ++++++++++++ . issue...,scan_copy_system,MALBEC PLUS YET1,Printing,US,Consumer
2,5080093007,2022-02-04 18:08:38,Log Type : Phone Log\r\nAction Type : Inbound ...,wh - done . wait time - done . prev exp - done...,ink_cartridge_print_quality,MALBEC I-INK,Printing,US,Consumer
3,5080093052,2022-02-04 18:21:37,Log Type : Notes Log \nAction Type : Inbou...,note type . order reference number from servic...,wont_power_on,Bib27PLI_20C2,Personal Systems,US,Consumer
4,5080093068,2022-02-10 17:52:33,Log Type: Customer Log&#xd;\nVisible Externall...,customernote desc: logged by: martelli paul ...,ink_cartridge_print_quality,ION 65,Printing,US,Commercial
...,...,...,...,...,...,...,...,...,...
659738,5080081042,2022-02-09 13:52:27,Log Type : Phone Log\r\nAction Type : Inbound ...,cs has printing issues . printer works fine ...,hardware_third_party,14 1C TSAIO ArchT Win8.1-64,Personal Systems,US,Consumer
659739,5080081048,2022-02-04 14:58:23,Log Type : Customer Log\r\nAction Type : Inbou...,problem description:unable to scan . troublesh...,scan_copy_system,MUSCATEL HI OJP I-INK,Printing,US,Consumer
659740,5080081061,2022-02-09 08:37:46,Notes Type : Notes Log\r\nIs Visible External ...,successfully submitted case . ==> . ts mail se...,internet_related,Picard,Personal Systems,US,Commercial
659741,5080081063,2022-02-10 23:41:01,Notes Type : Notes Log\r\nIs Visible External ...,successfully submitted case . ==> . setting up...,battery,Trypticon 14,Personal Systems,US,Commercial


In [8]:
query = f"""select rdr.case_id, 
                rdr.case_closed_date, 
                rdr.notes_history,
                rdr.notes_history_cleaned as notes_history_cleaned_v3,
                rdr.ip2, 
                rdr.platform_name, 
                rdr.business,
                rdr.country_code,
                rdr.segment
                --geo.Sub_Region1 as geo_market
            from {hlp.schema}.{rdr} rdr
            --left join {hlp.schema}.{geo_hier} geo
            --on rdr.country_code = geo.country_code 
            where date(case_closed_date) >= '{start_dat}'
            and business='Personal Systems' and segment='Commercial'
            and business is not null
            and platform_name is not null
            and notes_history is not null
            and ip2 is not null
            and language = 'en'
;"""

with hlp.get_connection() as conn:
    unprocessed = pd.read_sql(query, conn)
unprocessed

,case_id,case_closed_date,notes_history,notes_history_cleaned_v3,ip2,platform_name,business,country_code,segment
0,5075473161,2022-01-22 11:45:25,Log Type : Phone Log\r\nAction Type : Inbound ...,case subject: in nbd hp ids uma i u fwwan 64...,hinges,Cocoa 1.0,Personal Systems,IN,Commercial
1,5075473164,2021-12-04 03:11:59,Log Type : Phone Log\r\nAction Type : Inbound ...,abdulla puram village vellore . . problem de...,boot_to_black_screen,Louisa,Personal Systems,IN,Commercial
2,5075473184,2021-11-12 05:06:40,Notes Type : Notes Log\r\nIs Visible External ...,successfully submitted case . ==> . case subje...,keyboard,Cocoa 1.0,Personal Systems,IN,Commercial
3,5075473190,2021-11-14 10:05:03,Notes Type : Notes Log\r\nIs Visible External ...,正しく送信されたケース . ==> . case subject: hp probook...,keyboard,Ratchet,Personal Systems,IN,Commercial
4,5075473197,2021-11-17 08:18:49,Log Type : Notes Log\r\nVisible Externally : N...,. customer . . . covid-19 screening complet...,battery,Caesars14 1.0,Personal Systems,GB,Commercial
...,...,...,...,...,...,...,...,...,...
660621,5080092530,2022-02-17 18:17:31,Notes Type : Notes Log\r\nIs Visible External ...,successfully submitted case . ==> . bookable r...,configure_operating_system,Hagrid6U,Personal Systems,CO,Commercial
660622,5080092578,2022-02-16 14:04:25,Log Type : Phone Log\r\nAction Type : Inbound ...,problem description: adapter issue . -cx calle...,missing_item_in_box,Hornbill 1.0,Personal Systems,US,Commercial
660623,5080092584,2022-02-08 19:45:46,Log Type: Customer Log\r\nVisible Externally: ...,log type: customer log\n\nvisible externally: ...,user_accounts_passwords,Trypticon 14,Personal Systems,MX,Commercial
660624,5080092709,2022-02-11 13:18:17,Log Type : Notes Log\r\nAction Type : Inbound ...,greeting talk to agent . mxl kq . ==> . pc tu...,hard_drive,Trumpet,Personal Systems,US,Commercial


In [9]:
# hlp.update_from_another_table('EWS_PYATE_NOTES_RESTRUCTURED', )

In [9]:
# clean_sentence_wise()

cdax = CDAX()
# unprocessed['notes_history_cleaned_v3'] = unprocessed['notes_history_cleaned']

In [10]:
# Load cleaned to notes table

if len(unprocessed) > 0:
    hlp.analyse_df(unprocessed)
    hlp.load_to_vertica_database(unprocessed, hlp.schema, notes, chunk_size = 10000)
else:
    print('No unprocessed rows')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660626 entries, 0 to 660625
Data columns (total 9 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   case_id                   660626 non-null  int64         
 1   case_closed_date          660626 non-null  datetime64[ns]
 2   notes_history             660626 non-null  object        
 3   notes_history_cleaned_v3  660626 non-null  object        
 4   ip2                       660626 non-null  object        
 5   platform_name             660626 non-null  object        
 6   business                  660626 non-null  object        
 7   country_code              660611 non-null  object        
 8   segment                   660626 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 45.4+ MB


None

,case_id,case_closed_date,notes_history,notes_history_cleaned_v3,ip2,platform_name,business,country_code,segment
0,5075473161,2022-01-22 11:45:25,Log Type : Phone Log\r\nAction Type : Inbound ...,case subject: in nbd hp ids uma i u fwwan 64...,hinges,Cocoa 1.0,Personal Systems,IN,Commercial
1,5075473164,2021-12-04 03:11:59,Log Type : Phone Log\r\nAction Type : Inbound ...,abdulla puram village vellore . . problem de...,boot_to_black_screen,Louisa,Personal Systems,IN,Commercial
2,5075473184,2021-11-12 05:06:40,Notes Type : Notes Log\r\nIs Visible External ...,successfully submitted case . ==> . case subje...,keyboard,Cocoa 1.0,Personal Systems,IN,Commercial
3,5075473190,2021-11-14 10:05:03,Notes Type : Notes Log\r\nIs Visible External ...,正しく送信されたケース . ==> . case subject: hp probook...,keyboard,Ratchet,Personal Systems,IN,Commercial
4,5075473197,2021-11-17 08:18:49,Log Type : Notes Log\r\nVisible Externally : N...,. customer . . . covid-19 screening complet...,battery,Caesars14 1.0,Personal Systems,GB,Commercial


,case_id,case_closed_date,notes_history,notes_history_cleaned_v3,ip2,platform_name,business,country_code,segment
660621,5080092530,2022-02-17 18:17:31,Notes Type : Notes Log\r\nIs Visible External ...,successfully submitted case . ==> . bookable r...,configure_operating_system,Hagrid6U,Personal Systems,CO,Commercial
660622,5080092578,2022-02-16 14:04:25,Log Type : Phone Log\r\nAction Type : Inbound ...,problem description: adapter issue . -cx calle...,missing_item_in_box,Hornbill 1.0,Personal Systems,US,Commercial
660623,5080092584,2022-02-08 19:45:46,Log Type: Customer Log\r\nVisible Externally: ...,log type: customer log\n\nvisible externally: ...,user_accounts_passwords,Trypticon 14,Personal Systems,MX,Commercial
660624,5080092709,2022-02-11 13:18:17,Log Type : Notes Log\r\nAction Type : Inbound ...,greeting talk to agent . mxl kq . ==> . pc tu...,hard_drive,Trumpet,Personal Systems,US,Commercial
660625,5080092788,2022-02-11 17:32:47,GSEMCaseCreationNotes\r\nFalse\r\nAgreement De...,==end user support-osc . ==end user support-osc,office_suite,Catalonia 1.0,Personal Systems,US,Commercial


/home/ubuntu/Notebooks/DEV_ENV/lib/python3.7/site-packages/helper_pkg/__init__.py:128: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  display(df.describe(include='all'))


,case_id,case_closed_date,notes_history,notes_history_cleaned_v3,ip2,platform_name,business,country_code,segment
count,6.606260e+05,660626,660626,660626,660626,660626,660626,660611,660626
unique,NaN,641641,659768,573576,66,1633,1,192,1
top,NaN,2022-01-08 01:58:35,Log Type : Phone Log\r\nAction Type : Inbound ...,nps pass . end,battery,Computron 14,Personal Systems,IN,Commercial
freq,NaN,5,13,8795,82881,67601,660626,171410,660626
first,NaN,2021-11-01 00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2022-02-25 19:32:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,5.076773e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,3.286436e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,5.000052e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,5.075479e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Dividing provided dataframe into chunks of 10000 rows...
Number of chunks: 67
	Number of rows loaded: 9861
	Number of rows rejected: 0
	Number of rows loaded: 9861
	Number of rows rejected: 0
	Number of rows loaded: 9861
	Number of rows rejected: 0
	Number of rows loaded: 9861
	Number of rows rejected: 0
	Number of rows loaded: 9861
	Number of rows rejected: 0
	Number of rows loaded: 9861
	Number of rows rejected: 0
	Number of rows loaded: 9860
	Number of rows rejected: 0
	Number of rows loaded: 9860
	Number of rows rejected: 0
	Number of rows loaded: 9860
	Number of rows rejected: 0
	Number of rows loaded: 9860
	Number of rows rejected: 0
	Number of rows loaded: 9860
	Number of rows rejected: 0
	Number of rows loaded: 9860
	Number of rows rejected: 0
	Number of rows loaded: 9860
	Number of rows rejected: 0
	Number of rows loaded: 9860
	Number of rows rejected: 0
	Number of rows loaded: 9860
	Number of rows rejected: 0
	Number of rows loaded: 9860
	Number of rows rejected: 0
	Number of

### Term Extraction - PYATE

In [11]:
# apply function on the unprocessed data

from helper_pkg.modified_pyate import Modified_PYATE
from helper_pkg.pyate import PYATE
pyate = PYATE()

Pyate function imported


In [12]:
# t = 'great print quality. great print'
t = 'driver of m477.i will mail cust the link. . ==> . not contacted.'
res = Modified_PYATE.modified_combo_basic(t)
res

Series([], dtype: float64)

In [12]:
# q = f"""insert into pps_quality.{processed_table}
#         select * from pps_quality.EWS_PYATE_NOTES_PROCESSED_PRINT 
#         where case_id not in (select distinct case_id from pps_quality.{processed_table});
#         """
# hlp.run_query(q)

In [13]:
q = f"""select * from {hlp.schema}.{notes}
        where 
        --business = 'Personal Systems'
        {target_col} is not null
        and case_id not in (select distinct case_id from pps_quality.{processed_table})
        order by ip2, case_closed_date desc
        ;"""

with hlp.get_connection('aws3') as conn:
    df = pd.read_sql(q, conn)
    
df

,case_id,ip2,notes_history,case_closed_date,business,platform_name,country_code,notes_history_cleaned_v3,geo_market,segment
0,5080680655,ac_adapter,Log Type : Phone Log\r\nAction Type : Inbound ...,2022-02-25T19:31:52.000Z,Personal Systems,Redpine 1.0,BE,"adapter defective, 450 g7 . no previous cases ...",None,Commercial
1,5080806746,ac_adapter,Log Type: Customer Log&#xd;\nVisible Externall...,2022-02-25T19:25:55.000Z,Personal Systems,Computron 14,IN,customernote desc: charger issue . ==> . case ...,None,Commercial
2,5080762244,ac_adapter,Log Type : Phone Log\r\nAction Type : Inbound ...,2022-02-25T19:24:59.000Z,Personal Systems,Cadillac15 1.0,US,prepartrecommendationengine . dominant issue p...,None,Commercial
3,5080617651,ac_adapter,Log Type : Phone Log\r\nAction Type : Inbound ...,2022-02-25T19:22:02.000Z,Personal Systems,Potter6U,GB,david chan . + . cnd x . problem description:...,None,Commercial
4,5080610440,ac_adapter,** Automatically translated to English from [G...,2022-02-25T19:21:51.000Z,Personal Systems,Venus,DE,ac adapter failed . the ac adapter was tested ...,None,Commercial
...,...,...,...,...,...,...,...,...,...,...
171617,5078929953,wont_power_on,Notes Type : Notes Log\r\nIs Visible External ...,2022-01-17T22:05:24.000Z,Personal Systems,Scorpius 35W,AU,正しく送信されたケース . ==> . case title . problem des...,None,Commercial
171618,5078926299,wont_power_on,Notes Type : Notes Log\r\nIs Visible External ...,2022-01-17T22:03:51.000Z,Personal Systems,Crescent14 1.0,AU,successfully submitted case . ==> . prepartrec...,None,Commercial
171619,5078929160,wont_power_on,Log Type : Phone Log\r\nAction Type : Inbound ...,2022-01-17T21:15:01.000Z,Personal Systems,Tristar,HK,hk nbd csr psg no power - mainboard fail ctap ...,None,Commercial
171620,5078933043,wont_power_on,Log Type : Phone Log\r\nAction Type : Inbound ...,2022-01-17T20:46:55.000Z,Personal Systems,Oboe,AU,case title . au warr ons g2eoaio no po...,None,Commercial


In [24]:
remove_dict = {
            r'^=': '',
            r'^-': '',
            r'--+': ' ',
            r'===+': ' ',
            r'(\>\>|\=\>)': '.',
            r'\t+': ' ',
            r' +': ' ',
            r'\S+@\S+': ' ',
            r'\\+': ' ',
            r'/+': ' ',
            r'#+': '',
            r'\s•\s': ' ',
            r'[_\'\"-\(\)\^|]+': '',
            r';+': '.',
            r'[<\[\]]+': '',
            r'[\{\}]+': '',
            r'- -': '',
            r'\d:\d\d (a|p)m [a-z]*:': '',
            r'\d:\d\d (a|p)m': '',
            r'\r': ' ',
            r'\*+': '',
            r'[\d\-]{4,12}': ' '
        }

In [28]:
excel=pd.read_excel(r'C:\Users/koler/Documents/CS Advanced Analytics/Weak Signals/Win11/keyword_for_text_cleaning.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/koler/Documents/CS Advanced Analytics/Weak Signals/Win11/keyword_for_text_cleaning.xlsx'

In [ ]:
issue_string=set([i.lower().strip() for i in excel.Issue.dropna().astype(str).values.tolist()])
action_string=set([i.lower().strip() for i in excel.Action.dropna().astype(str).values.tolist()])
soln=["resolution"]
post_clean_str=[i.lower() for i in excel.Remove_Words_from_Cleaned_Texts.dropna().astype(str).values.tolist()]

In [14]:
pyate.apply(df, target_col, id_col, processed_table, block_size=50000, verbose=False)

  0%|          | 0/50000 [00:00<?, ?it/s]

Total blocks: 4


Block: 1
Range: 0:50000


  0%|          | 0/50000 [00:00<?, ?it/s]

	Number of rows loaded: 50000
	Number of rows rejected: 0


Total Rows loaded: 50000
Total rejected rows: 0
Block: 2
Range: 50000:100000


  0%|          | 0/50000 [00:00<?, ?it/s]

	Number of rows loaded: 50000
	Number of rows rejected: 0


Total Rows loaded: 50000
Total rejected rows: 0
Block: 3
Range: 100000:150000


  0%|          | 2/21622 [00:00<20:04, 17.95it/s]

	Number of rows loaded: 50000
	Number of rows rejected: 0


Total Rows loaded: 50000
Total rejected rows: 0
Block: 4
Range: 150000:171622


100%|██████████| 21622/21622 [06:22<00:00, 56.57it/s]


	Number of rows loaded: 21622
	Number of rows rejected: 0


Total Rows loaded: 21622
Total rejected rows: 0


In [15]:
print('complete')

complete


In [16]:
q = f"""select * from pps_quality.{processed_table}
        where case_id not in (select distinct case_id from pps_quality.{restructured_table})
        and term_1 is not null
        and term_1 <> 'nan';
        """

result, _ = pyate.get_data(q, chunksize=1000000000000)
result

Final record_count: 157047


,case_id,ip2,notes_history,case_closed_date,business,platform_name,country_code,notes_history_cleaned_v3,geo_market,term_1,...,term_18,score_18,count_18,term_19,score_19,count_19,term_20,score_20,count_20,segment
0,5036908304,boot_to_black_screen,Notes Type : Notes Log\r\nIs Visible External ...,2022-02-25T11:17:51.000Z,Personal Systems,Bugle,FJ,successfully submitted case . ==> . 9 11 . s...,None,power reset powersupply,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Commercial
1,5047193311,boot_to_black_screen,Log Type : Notes Log\r\nVisible Externally : N...,2022-01-28T09:15:22.000Z,Personal Systems,Ratchet,GB,"my laptop is on, i can hear the the fans turn...",None,bios update screen displays,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Commercial
2,5049596063,bios,Log Type : Notes Log\r\nVisible Externally : N...,2022-01-28T09:26:16.000Z,Personal Systems,Caesars15 1.0,GB,as soon as you turn the laptop on it goes into...,None,serial number- 5cg gy2,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Commercial
3,5049718453,battery,Notes Type : Notes Log\r\nIs Visible External ...,2022-01-25T01:04:02.000Z,Personal Systems,Potter6U,FJ,successfully submitted case . ==> . hi savnil ...,None,power ac adaptor,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Commercial
4,5050409422,bios,Log Type : Customer Log\r\nAction Type : Inbou...,2022-02-10T11:19:46.000Z,Personal Systems,Clarinet,US,session id -> . interaction type -> chat . no...,None,additional address details updation,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Commercial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157042,5081223399,internal_display,** Automatically translated to English from [G...,2022-02-25T09:52:14.000Z,Personal Systems,Redpine 1.0,CH,screen is broken has caught some printing poin...,None,printing points the accident,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Commercial
157043,5081227554,keyboard,Log Type : Phone Log\r\nAction Type : Inbound ...,2022-02-25T10:50:12.000Z,Personal Systems,Potter6U,GR,problem description: issue with keyboard keys,None,keyboard keys,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Commercial
157044,5081233754,internal_display,Log Type : Phone Log\r\nAction Type : Inbound ...,2022-02-25T12:18:34.000Z,Personal Systems,Tortilla6U 4.0,US,verified caller info . validated warranty deta...,None,few black spots,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Commercial
157045,5081237799,internet_related,Log Type : Phone Log\r\nAction Type : Inbound ...,2022-02-25T13:27:39.000Z,Personal Systems,Excalibur,US,"cu is trying to send email , gets error invali...",None,error invalid recipient,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Commercial


In [6]:
# processed='EWS_WIN11_PYATE123'

In [10]:
# q = f"""select * from pps_quality.{processed}
#         where term_1 is not null
#         and term_1 <> 'nan';
#         """

# result, _ = pyate.get_data(q, chunksize=1000000000000)
# result

Final record_count: 53415


,case_id,case_closed_date,notes_history,notes_history_cleaned_v3,ip2,platform_name,business,country_code,segment,problem_description,...,count_17,term_18,score_18,count_18,term_19,score_19,count_19,term_20,score_20,count_20
0,5030431332,2021-10-31 06:46:24,Log Type : Customer Log\r\nAction Type : Inbou...,problem description: . rirvco connectivity iss...,web_services,VERONA I-INK,Printing,US,Consumer,.\nrirvco connectivity issue,...,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN
1,5034955775,2021-12-02 06:59:45,Log Type : Customer Log\r\nAction Type : Inbou...,top category : post setup related issue . ==> ...,sw_installation,MUSCATEL OJP I-INK,Printing,CA,Consumer,cant print,...,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN
2,5037434513,2021-11-02 01:10:51,Log Type : Customer Log\r\nAction Type : Inbou...,problem description: older customer calls indi...,scan_copy_system,VERONA I-INK,Printing,ES,Consumer,older customer calls indicating that they have...,...,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN
3,5045634493,2021-11-23 20:10:16,Log Type : Customer Log\r\nAction Type : Inbou...,problem description: cannot print issue . ==> ...,sw_installation,SHAOLIN PLUS I-INK,Printing,IT,Consumer,cannot print issue,...,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN
4,5052807864,2021-11-25 16:41:00,Log Type : Customer Log\r\nAction Type : Inbou...,case subject: cu enur in nsr 8gb02pa win10 ...,touchpad,Faroe_19C2,Personal Systems,IN,Consumer,cu enur in nsr 8gb02pa win10 v. hardware i...,...,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53410,5076930746,2021-12-05 20:17:31,Log Type : Phone Log\r\nAction Type : Inbound ...,problem description: driver and printhead issu...,ink_cartridge_print_quality,POSEIDON WL,Printing,TR,Consumer,driver and printhead issue,...,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN
53411,5076930758,2021-12-05 20:10:05,Log Type : Phone Log\r\nAction Type : Inbound ...,problem description: cant print from pc . wire...,sw_installation,MALBEC I-INK,Printing,IL,Consumer,cant print from pc,...,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN
53412,5076930769,2021-12-05 19:52:41,Log Type : Phone Log\r\nAction Type : Inbound ...,problem description: connect to the wifi direc...,sw_installation,RATCHET MFP,Printing,IL,Consumer,connect to the wifi direct,...,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN
53413,5076931118,2021-12-05 19:50:44,Log Type : Phone Log\r\nAction Type : Inbound ...,issue: . no power . cu claims pc cant power ...,wont_power_on,Reeds 1.0,Personal Systems,IT,Commercial,- no power,...,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN


In [17]:
result.drop_duplicates(inplace=True, ignore_index=True)

In [18]:
result['notes_history_cleaned_v3'] = np.nan

In [19]:
# restructure the processed data
result = pyate.restructure(
                            result, 'case_id', 'notes', 
                           ['case_id', 'case_closed_date', 'ip2', 'business', 
                                'platform_name', 'country_code','geo_market', 'segment']
                          )
result

,case_id,case_closed_date,ip2,business,platform_name,country_code,geo_market,segment,term,score,count,word_length
0,5036908304,2022-02-25T11:17:51.000Z,boot_to_black_screen,Personal Systems,Bugle,FJ,None,Commercial,power reset powersupply,1.443147,1.0,3
1,5047193311,2022-01-28T09:15:22.000Z,boot_to_black_screen,Personal Systems,Ratchet,GB,None,Commercial,bios update screen displays,3.636294,3.0,4
2,5049596063,2022-01-28T09:26:16.000Z,bios,Personal Systems,Caesars15 1.0,GB,None,Commercial,serial number- 5cg gy2,2.193147,1.0,4
3,5049718453,2022-01-25T01:04:02.000Z,battery,Personal Systems,Potter6U,FJ,None,Commercial,power ac adaptor,3.495837,3.0,3
4,5050409422,2022-02-10T11:19:46.000Z,bios,Personal Systems,Clarinet,US,None,Commercial,additional address details updation,2.943147,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...
3136649,5080813260,2022-02-20T04:02:58.000Z,internal_display,Personal Systems,Pelican14 1.0,IN,None,Commercial,bios mode,0.693147,1.0,2
3138915,5080866966,2022-02-23T08:03:17.000Z,hangs_at_p.o.s.t,Personal Systems,Pelican14 1.0,IN,None,Commercial,power drain,1.386294,2.0,2
3139280,5081009135,2022-02-23T12:22:21.000Z,internal_display,Personal Systems,Oleander 1.0,GB,None,Commercial,physical damage,0.693147,1.0,2
3140092,5081137985,2022-02-25T02:59:20.000Z,internal_display,Personal Systems,Ramia 1.0,LK,None,Commercial,battery issue,1.386294,2.0,2


In [14]:
# restructured='EWS_WIN11_Lemmatized_restructure'

In [20]:
hlp.load_to_vertica_database(result, hlp.schema, restructured_table,chunk_size = 100000)

Dividing provided dataframe into chunks of 100000 rows...
Number of chunks: 11
	Number of rows loaded: 99745
	Number of rows rejected: 0
	Number of rows loaded: 99745
	Number of rows rejected: 0
	Number of rows loaded: 99745
	Number of rows rejected: 0
	Number of rows loaded: 99745
	Number of rows rejected: 0
	Number of rows loaded: 99745
	Number of rows rejected: 0
	Number of rows loaded: 99745
	Number of rows rejected: 0
	Number of rows loaded: 99745
	Number of rows rejected: 0
	Number of rows loaded: 99745
	Number of rows rejected: 0
	Number of rows loaded: 99744
	Number of rows rejected: 0
	Number of rows loaded: 99744
	Number of rows rejected: 0
	Number of rows loaded: 99744
	Number of rows rejected: 0


Total Rows loaded: 1097192
Total rejected rows: 0


'Successful'

In [21]:
q = f"""select term, count(*)
        from pps_quality.{restructured_table}
        where term not in (select distinct term from pps_quality.EWS_LEMMAS)
        group by term
        having count(*) >= 30;"""

terms_df = hlp.get_sample_data(q, rows=1000000000)
hlp.analyse_df(terms_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   term    141 non-null    object
 1   count   141 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.3+ KB


None

,term,count
0,ncsr battery,30
1,dec’19 maturity devices,35
2,hardreset- issue,44
3,onsite confirmation email,36
4,endpoint engineer,32


,term,count
136,pc aio display issue,38
137,bare minimum boot issue,30
138,warranty part sales offer,38
139,trial mail processing case,31
140,cheque paynow,30


,term,count
count,141,141.000000
unique,141,NaN
top,many connection issues,NaN
freq,1,NaN
mean,NaN,38.141844
std,NaN,21.677698
min,NaN,30.000000
25%,NaN,30.000000
50%,NaN,33.000000
75%,NaN,36.000000


In [ ]:
# with hlp.get_connection() as conn:
#     terms_df = pd.read_sql("""select * from pps_quality.EWS_LEMMAS;""", conn)
# terms_df

In [22]:
# Lemmatize terms

if len(terms_df) > 0:
    import spacy
    spacy.prefer_gpu()
    nlp = spacy.load("en_core_web_lg", disable=['ner', 'tagger', 'parser'])

    lemmatized_map = {}

    exclusion_list = ['bios', 'pending', 'number']

    correction_map = {'isues':'issues',
                      'issuess': 'issues',
                    'adapter':'adaptor',
                    'dispaly':'display',
                    'kbd':'keyboard',
                    'ts': 'troubleshooting',
                    'dsply': 'display',
                    'imcompatible':'incompatible',
                    
                    }

    corrected_terms = []
    terms = terms_df.term.unique()
    for term in terms:
        words = term.split()
        temp_term = []
        for word in words:
            if word in correction_map.keys():
                word = correction_map[word]

            temp_term.append(word)
        corrected_terms.append(' '.join(temp_term))

    print(f'Total terms: {len(terms)}')

    for i, doc in enumerate(nlp.pipe(corrected_terms)):
        if i % 100000 == 0:
            print(f'Completed {i}')        
        lemmas = []
        for word in doc:
            if word.text in exclusion_list:
                lemmas.append(word.text)
            else:
                lemmas.append(word.lemma_)

        lemmatized_map.update({doc.text: ' '.join(lemmas)})

    lemmatized_map
    terms_df['lemmatized'] = terms_df.term.map(lemmatized_map)
    display(terms_df)

Total terms: 141
Completed 0


,term,count,lemmatized
0,ncsr battery,30,ncsr battery
1,dec’19 maturity devices,35,dec’19 maturity device
2,hardreset- issue,44,hardreset- issue
3,onsite confirmation email,36,onsite confirmation email
4,endpoint engineer,32,endpoint engineer
...,...,...,...
136,pc aio display issue,38,pc aio display issue
137,bare minimum boot issue,30,bare minimum boot issue
138,warranty part sales offer,38,warranty part sale offer
139,trial mail processing case,31,trial mail process case


In [23]:
terms_df['lemmatized'] = terms_df.term.map(lemmatized_map)

In [24]:
terms_df

,term,count,lemmatized
0,ncsr battery,30,ncsr battery
1,dec’19 maturity devices,35,dec’19 maturity device
2,hardreset- issue,44,hardreset- issue
3,onsite confirmation email,36,onsite confirmation email
4,endpoint engineer,32,endpoint engineer
...,...,...,...
136,pc aio display issue,38,pc aio display issue
137,bare minimum boot issue,30,bare minimum boot issue
138,warranty part sales offer,38,warranty part sale offer
139,trial mail processing case,31,trial mail process case


In [25]:
# hlp.run_query("""drop table pps_quality.EWS_LEMMAS;""")
hlp.load_to_vertica_database(terms_df, hlp.schema, 'EWS_LEMMAS')

	Number of rows loaded: 141
	Number of rows rejected: 0


Total Rows loaded: 141
Total rejected rows: 0


'Successful'

In [26]:
result['lemmatized'] = result['term'].map(lemmatized_map)

In [27]:
result

,case_id,case_closed_date,ip2,business,platform_name,country_code,geo_market,segment,term,score,count,word_length,lemmatized
0,5036908304,2022-02-25T11:17:51.000Z,boot_to_black_screen,Personal Systems,Bugle,FJ,None,Commercial,power reset powersupply,1.443147,1.0,3,NaN
1,5047193311,2022-01-28T09:15:22.000Z,boot_to_black_screen,Personal Systems,Ratchet,GB,None,Commercial,bios update screen displays,3.636294,3.0,4,NaN
2,5049596063,2022-01-28T09:26:16.000Z,bios,Personal Systems,Caesars15 1.0,GB,None,Commercial,serial number- 5cg gy2,2.193147,1.0,4,NaN
3,5049718453,2022-01-25T01:04:02.000Z,battery,Personal Systems,Potter6U,FJ,None,Commercial,power ac adaptor,3.495837,3.0,3,NaN
4,5050409422,2022-02-10T11:19:46.000Z,bios,Personal Systems,Clarinet,US,None,Commercial,additional address details updation,2.943147,1.0,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136649,5080813260,2022-02-20T04:02:58.000Z,internal_display,Personal Systems,Pelican14 1.0,IN,None,Commercial,bios mode,0.693147,1.0,2,NaN
3138915,5080866966,2022-02-23T08:03:17.000Z,hangs_at_p.o.s.t,Personal Systems,Pelican14 1.0,IN,None,Commercial,power drain,1.386294,2.0,2,NaN
3139280,5081009135,2022-02-23T12:22:21.000Z,internal_display,Personal Systems,Oleander 1.0,GB,None,Commercial,physical damage,0.693147,1.0,2,NaN
3140092,5081137985,2022-02-25T02:59:20.000Z,internal_display,Personal Systems,Ramia 1.0,LK,None,Commercial,battery issue,1.386294,2.0,2,NaN


In [28]:
hlp.update_from_another_table(restructured_table, 'EWS_LEMMAS',
                             'term',
                             ['lemmatized'],
                             update_only_nulls=True)

Running 
update pps_quality.EWS_PYATE_NOTES_RESTRUCTURED tgt
                set lemmatized = src.lemmatized
                from pps_quality.EWS_LEMMAS src
                where tgt.term = src.term
             and (tgt.lemmatized is null)
 on aws3
Ran successfully!


In [31]:
hlp.update_from_another_table(restructured_table, 'RDR_MASTER',
                             'case_id',
                             ['ip1','ip3'],
                             update_only_nulls=True)

Running 
update pps_quality.EWS_PYATE_NOTES_RESTRUCTURED tgt
                set ip1 = src.ip1, ip3 = src.ip3
                from pps_quality.RDR_MASTER src
                where tgt.case_id = src.case_id
             and (tgt.ip1 is null or tgt.ip3 is null)
 on aws3
Failed!
Due to Severity: ERROR, Message: Column tgt.ip1 does not exist, Sqlstate: 42703, Routine: unknown_attribute, File: /scratch_a/release/svrtar18593/vbuild/vertica/Parser/ParseFunc.cpp, Line: 1768, Error Code: 2656, SQL: 'update pps_quality.EWS_PYATE_NOTES_RESTRUCTURED tgt                 set ip1 = src.ip1, ip3 = src.ip3                 from pps_quality.RDR_MASTER src                 where tgt.case_id = src.case_id              and (tgt.ip1 is null or tgt.ip3 is null)'


In [64]:
# hlp.add_columns_to_table('EWS_PYATE_NOTES_RESTRUCTURED_NEW', {'lemmatized': 'varchar(1000)'})

In [32]:
# q = f"""update pps_quality.EWS_PYATE_NOTES_RESTRUCTURED tgt
#                 set lemmatized = src.lemmatized
#                 from pps_quality.EWS_LEMMAS src
#                 where tgt.term = src.term
#                 and tgt.lemmatized is null;"""

hlp.update_from_another_table(restructured_table, 'EWS_LEMMAS',
                             'term',
                             ['lemmatized'],
                             update_only_nulls=True)

# hlp.run_query(q)

Running 
update pps_quality.EWS_PYATE_NOTES_RESTRUCTURED tgt
                set lemmatized = src.lemmatized
                from pps_quality.EWS_LEMMAS src
                where tgt.term = src.term
             and (tgt.lemmatized is null)
 on aws3
Ran successfully!


In [33]:
hlp.update_from_another_table(restructured_table, 'RDR_MASTER',
                             'case_id',
                             ['product_category', 'geo_market'],
                             update_only_nulls=True)

Running 
update pps_quality.EWS_PYATE_NOTES_RESTRUCTURED tgt
                set product_category = src.product_category, geo_market = src.geo_market
                from pps_quality.RDR_MASTER src
                where tgt.case_id = src.case_id
             and (tgt.product_category is null or tgt.geo_market is null)
 on aws3
Ran successfully!


### Term Prioritisation - RIDF

In [34]:
# segment='CONSUMER'
segment = 'COMMERCIAL'

In [35]:
q = """select max(date(case_closed_date)) from pps_quality.EWS_PYATE_NOTES_RESTRUCTURED;"""
max_date = hlp.get_sample_data(q)
max_date = max_date['max'].values[0]


print(max_date)

2022-02-25


In [41]:
cutoff_obj = (max_date - timedelta(days=55))

cutoff_date = cutoff_obj.strftime('%Y-%m-%d')
end_date = (cutoff_obj + timedelta(days=55)).strftime('%Y-%m-%d')


print(cutoff_date)
print(end_date)

2022-01-01
2022-02-25


In [42]:
q = f"""select lemmatized, count(distinct case_id) from pps_quality.{restructured_table} 
        
        where date(case_closed_date) >= '{cutoff_date}'
        and date(case_closed_date) <= '{end_date}'
        
        --where date(case_closed_date) >= '2020-12-15'
        --and date(case_closed_date) <= '2020-12-31'
        
        and lemmatized is not null
        and lower(segment) = '{segment.lower()}'
        group by lemmatized
        having count(distinct case_id) >= 30
        order by count(*) desc;
        """
valid_terms = hlp.get_sample_data(q, rows=10000000)
terms = valid_terms.lemmatized.unique()
print(f'Total terms: {len(terms)}')
terms_str = "', '".join(terms)

Total terms: 6368


In [43]:
q = f"""select * from pps_quality.{restructured_table}
        where date(case_closed_date) >= '{cutoff_date}'
        and date(case_closed_date) <= '{end_date}'
        
        --where date(case_closed_date) >= '2020-12-15'
        --and date(case_closed_date) <= '2020-12-31'
        
        and lemmatized in (
            '{terms_str}'
        );
        """

with hlp.get_connection() as aws_conn:
    df = pd.read_sql(q, aws_conn)
df

,case_id,case_closed_date,ip2,business,platform_name,country_code,geo_market,term,score,count,word_length,lemmatized,product_category,end_user_company_name,serial_number,product,Segment,Sub_Reg,Region
0,5070178381,2022-01-06T15:15:18.000Z,battery,Personal Systems,Computron 14,US,North America,swollen battery,1.386294,2.0,2,swell battery,Commercial Notebooks,Andrew Fertal,5CG9341LN9,2FA58AV,Commercial,None,None
1,5070178442,2022-01-06T15:14:50.000Z,perform_recovery,Personal Systems,Baritone-R,US,North America,cloud recovery,1.443147,1.0,2,cloud recovery,Commercial Desktops,oleg Domovsky,MXL0343B1B,9HT43UC,Commercial,None,None
2,5070178442,2022-01-06T15:14:50.000Z,perform_recovery,Personal Systems,Baritone-R,US,North America,flash drive,0.693147,1.0,2,flash drive,Commercial Desktops,oleg Domovsky,MXL0343B1B,9HT43UC,Commercial,None,None
3,5070178442,2022-01-06T15:14:50.000Z,perform_recovery,Personal Systems,Baritone-R,US,North America,pro std,0.693147,1.0,2,pro std,Commercial Desktops,oleg Domovsky,MXL0343B1B,9HT43UC,Commercial,None,None
4,5070178442,2022-01-06T15:14:50.000Z,perform_recovery,Personal Systems,Baritone-R,US,North America,recovery option,2.136294,2.0,2,recovery option,Commercial Desktops,oleg Domovsky,MXL0343B1B,9HT43UC,Commercial,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2841824,5081247387,2022-02-25T16:57:32.000Z,warranty_entitlement,Personal Systems,Lanyu,US,North America,folder icon,1.386294,2.0,2,folder icon,Workstations,None,None,None,Commercial,None,None
2841825,5081247451,2022-02-25T17:37:15.000Z,touchscreen,Personal Systems,Dolphin 1.0,US,North America,several times,0.693147,1.0,2,several time,Commercial Notebooks,None,None,None,Commercial,None,None
2841826,5081248872,2022-02-25T17:48:11.000Z,sleep_hibernate,Personal Systems,KINGS ELVIS,US,North America,external devices,0.693147,1.0,2,external device,Commercial Displays,None,None,None,Commercial,None,None
2841827,5081248872,2022-02-25T17:48:11.000Z,sleep_hibernate,Personal Systems,KINGS ELVIS,US,North America,hardware issue,0.693147,1.0,2,hardware issue,Commercial Displays,None,None,None,Commercial,None,None


In [44]:
ps = df.loc[df.business == 'Personal Systems', :].copy()
#prnt = df.loc[df.business == 'Printing', :].copy()

# del df

In [45]:
ps
#prnt

,case_id,case_closed_date,ip2,business,platform_name,country_code,geo_market,term,score,count,word_length,lemmatized,product_category,end_user_company_name,serial_number,product,Segment,Sub_Reg,Region
0,5070178381,2022-01-06T15:15:18.000Z,battery,Personal Systems,Computron 14,US,North America,swollen battery,1.386294,2.0,2,swell battery,Commercial Notebooks,Andrew Fertal,5CG9341LN9,2FA58AV,Commercial,None,None
1,5070178442,2022-01-06T15:14:50.000Z,perform_recovery,Personal Systems,Baritone-R,US,North America,cloud recovery,1.443147,1.0,2,cloud recovery,Commercial Desktops,oleg Domovsky,MXL0343B1B,9HT43UC,Commercial,None,None
2,5070178442,2022-01-06T15:14:50.000Z,perform_recovery,Personal Systems,Baritone-R,US,North America,flash drive,0.693147,1.0,2,flash drive,Commercial Desktops,oleg Domovsky,MXL0343B1B,9HT43UC,Commercial,None,None
3,5070178442,2022-01-06T15:14:50.000Z,perform_recovery,Personal Systems,Baritone-R,US,North America,pro std,0.693147,1.0,2,pro std,Commercial Desktops,oleg Domovsky,MXL0343B1B,9HT43UC,Commercial,None,None
4,5070178442,2022-01-06T15:14:50.000Z,perform_recovery,Personal Systems,Baritone-R,US,North America,recovery option,2.136294,2.0,2,recovery option,Commercial Desktops,oleg Domovsky,MXL0343B1B,9HT43UC,Commercial,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2841824,5081247387,2022-02-25T16:57:32.000Z,warranty_entitlement,Personal Systems,Lanyu,US,North America,folder icon,1.386294,2.0,2,folder icon,Workstations,None,None,None,Commercial,None,None
2841825,5081247451,2022-02-25T17:37:15.000Z,touchscreen,Personal Systems,Dolphin 1.0,US,North America,several times,0.693147,1.0,2,several time,Commercial Notebooks,None,None,None,Commercial,None,None
2841826,5081248872,2022-02-25T17:48:11.000Z,sleep_hibernate,Personal Systems,KINGS ELVIS,US,North America,external devices,0.693147,1.0,2,external device,Commercial Displays,None,None,None,Commercial,None,None
2841827,5081248872,2022-02-25T17:48:11.000Z,sleep_hibernate,Personal Systems,KINGS ELVIS,US,North America,hardware issue,0.693147,1.0,2,hardware issue,Commercial Displays,None,None,None,Commercial,None,None


In [47]:
def get_dictionary(business):        
    q = f"""select * from {hlp.schema}.{dictionaries[business]};"""
    
    dictionary, _ = hlp.get_data(q)
    return dictionary

ps_dictionary = get_dictionary('ps')
#print_dictionary = get_dictionary('print')
ps_dictionary

Final record_count: 34703


,Unnamed: 0,Unnamed: 0.1,term,type,theme,topic,count,L2 theme,L1 theme
0,0.0,0.0,ac adpater,specific,ac power cord,None,48.0,ac adaptor,power
1,2.0,3.0,audio jack issue,specific,audio issue,None,43.0,audio issue,button
2,3.0,8.0,battery calibration,specific,calibration issue,battery issue,NaN,battery calibration,battery
3,8.0,28.0,computer freeze,specific,computer freeze,None,162.0,computer freeze,firmware/software/driver
4,10.0,32.0,damage device,generic,damage device,damage,NaN,damage device,other
...,...,...,...,...,...,...,...,...,...
34698,NaN,NaN,﻿1 call,None,None,None,100.0,None,None
34699,NaN,NaN,﻿1 interaction,None,None,None,12.0,None,None
34700,NaN,NaN,﻿1 name,None,None,None,1.0,None,None
34701,NaN,NaN,﻿1 thing,None,None,None,60.0,None,None


In [48]:
temp = ps_dictionary.drop_duplicates(subset=['term'])
#temp = print_dictionary.drop_duplicates(subset=['term'])
len(temp)

14597

In [79]:
# q = """select copy_table('pps_quality.EWS_DICTIONARY_PS_WITH_MISSING', 'pps_quality.EWS_DICTIONARY_PS_UPDATED')"""
# hlp.run_query(q)

In [80]:
# q = """drop table pps_quality.EWS_DICTIONARY_PS_UPDATED;"""
# hlp.run_query(q)

In [81]:
# hlp.load_to_vertica_database(temp, hlp.schema, 'EWS_DICTIONARY_PS_UPDATED')

In [49]:
test = ps_dictionary.drop_duplicates(subset=['term'], keep='first')
#test = print_dictionary.drop_duplicates(subset=['term'], keep='first')
#test.drop(columns = ['Unnamed: 7','Unnamed: 0.1','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11'],inplace = True)
hlp.analyse_df(test)
ps_dictionary.groupby(['term']).agg(term_count = ('type', 'count'))
#print_dictionary.groupby(['term']).agg(term_count = ('type', 'count'))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14597 entries, 0 to 34701
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    8230 non-null   float64
 1   Unnamed: 0.1  7539 non-null   float64
 2   term          14596 non-null  object 
 3   type          8229 non-null   object 
 4   theme         1712 non-null   object 
 5   topic         337 non-null    object 
 6   count         14249 non-null  float64
 7   L2 theme      1405 non-null   object 
 8   L1 theme      1405 non-null   object 
dtypes: float64(3), object(6)
memory usage: 1.1+ MB


None

,Unnamed: 0,Unnamed: 0.1,term,type,theme,topic,count,L2 theme,L1 theme
0,0.0,0.0,ac adpater,specific,ac power cord,None,48.0,ac adaptor,power
1,2.0,3.0,audio jack issue,specific,audio issue,None,43.0,audio issue,button
2,3.0,8.0,battery calibration,specific,calibration issue,battery issue,NaN,battery calibration,battery
3,8.0,28.0,computer freeze,specific,computer freeze,None,162.0,computer freeze,firmware/software/driver
4,10.0,32.0,damage device,generic,damage device,damage,NaN,damage device,other


,Unnamed: 0,Unnamed: 0.1,term,type,theme,topic,count,L2 theme,L1 theme
34694,NaN,NaN,z2g4 mini p ws18,None,None,None,94.0,None,None
34695,NaN,NaN,zbook studio,None,None,None,85.0,None,None
34697,NaN,NaN,﻿1 bios,None,None,None,35.0,None,None
34698,NaN,NaN,﻿1 call,None,None,None,100.0,None,None
34701,NaN,NaN,﻿1 thing,None,None,None,60.0,None,None


,Unnamed: 0,Unnamed: 0.1,term,type,theme,topic,count,L2 theme,L1 theme
count,8230.000000,7539.000000,14596,8229,1712,337,14249.000000,1405,1405
unique,NaN,NaN,14596,2,479,77,NaN,549,25
top,NaN,NaN,privacy statement,generic,swollen battery,others,NaN,swollen battery,battery
freq,NaN,NaN,1,6859,75,33,NaN,86,176
mean,6416.719077,3982.497546,NaN,NaN,NaN,NaN,233.656046,NaN,NaN
std,4935.408080,2314.075406,NaN,NaN,NaN,NaN,1690.747737,NaN,NaN
min,0.000000,0.000000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN
25%,2167.250000,2005.500000,NaN,NaN,NaN,NaN,30.000000,NaN,NaN
50%,4736.500000,3959.000000,NaN,NaN,NaN,NaN,51.000000,NaN,NaN
75%,10670.750000,5952.500000,NaN,NaN,NaN,NaN,106.000000,NaN,NaN


,term_count
term,
-cdax case,0
-no answer,0
-no go,0
-no issue,0
-same issue,0
...,...
﻿1 time,0
﻿1 time issue,0
﻿1 time setup,0


In [83]:
# ps_dictionary.to_excel('output/ps_dictionary_2020_12_04_db.xlsx', index=False)

In [50]:
# PS
ps_merged = pd.merge(ps, ps_dictionary, how='left', left_on='lemmatized', right_on = 'term')
relevant_ps = ps_merged.loc[(~ps_merged['type'].isin(['stopwords', 'default'])) | (ps_merged['type'].isnull())]
relevant_ps

# # PRINT
# print_merged = pd.merge(prnt, print_dictionary, how='left', left_on='lemmatized', right_on = 'term')
# relevant_print = print_merged.loc[(~print_merged['type'].isin(['stopwords', 'default'])) | (print_merged['type'].isnull())]
# relevant_print

,case_id,case_closed_date,ip2,business,platform_name,country_code,geo_market,term_x,score,count_x,...,Region,Unnamed: 0,Unnamed: 0.1,term_y,type,theme,topic,count_y,L2 theme,L1 theme
0,5070178381,2022-01-06T15:15:18.000Z,battery,Personal Systems,Computron 14,US,North America,swollen battery,1.386294,2.0,...,None,9744.0,258.0,swell battery,generic,screen error,screen issue,NaN,screen error,display
1,5070178442,2022-01-06T15:14:50.000Z,perform_recovery,Personal Systems,Baritone-R,US,North America,cloud recovery,1.443147,1.0,...,None,9988.0,1022.0,cloud recovery,specific,cloud recovery,None,65.0,None,None
2,5070178442,2022-01-06T15:14:50.000Z,perform_recovery,Personal Systems,Baritone-R,US,North America,flash drive,0.693147,1.0,...,None,594.0,1744.0,flash drive,specific,flash driver,None,14.0,flash driver,firmware/software/driver
3,5070178442,2022-01-06T15:14:50.000Z,perform_recovery,Personal Systems,Baritone-R,US,North America,pro std,0.693147,1.0,...,None,1348.0,4108.0,pro std,generic,None,None,424.0,None,None
4,5070178442,2022-01-06T15:14:50.000Z,perform_recovery,Personal Systems,Baritone-R,US,North America,recovery option,2.136294,2.0,...,None,1381.0,4245.0,recovery option,generic,None,None,14.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7756491,5081248872,2022-02-25T17:48:11.000Z,sleep_hibernate,Personal Systems,KINGS ELVIS,US,North America,hardware issue,0.693147,1.0,...,None,9678.0,95.0,hardware issue,specific,hardware issue,None,71.0,fingerprint scanner,io
7756492,5081248872,2022-02-25T17:48:11.000Z,sleep_hibernate,Personal Systems,KINGS ELVIS,US,North America,hardware issue,0.693147,1.0,...,None,12806.0,9285.0,hardware issue,specific,hardware issue,None,30.0,hardware issue,other
7756493,5081248872,2022-02-25T17:48:11.000Z,sleep_hibernate,Personal Systems,KINGS ELVIS,US,North America,power drain,0.693147,1.0,...,None,3028.0,9115.0,power drain,specific,power drain,None,50.0,power drain,power
7756494,5081248872,2022-02-25T17:48:11.000Z,sleep_hibernate,Personal Systems,KINGS ELVIS,US,North America,power drain,0.693147,1.0,...,None,10295.0,1947.0,power drain,specific,power drain,None,88.0,power drain,power


In [51]:
#missing_terms_seg = pd.DataFrame(relevant_print[relevant_print.type.isnull()].lemmatized.value_counts())
missing_terms_seg = pd.DataFrame(relevant_ps[relevant_ps.type.isnull()].lemmatized.value_counts())

missing_terms_seg.reset_index(inplace=True)
missing_terms_seg.columns = ['term', 'count']
missing_terms_seg

,term,count
0,problem description,990990
1,workorder summary,353010
2,order reference number,317229
3,service order,300245
4,main topic issue,178695
...,...,...
2601,issue problem description field,1
2602,different tray,1
2603,separation pad,1
2604,advance clean kit,1


In [44]:
hlp.load_to_vertica_database(missing_terms_seg, hlp.schema, 'EWS_DICTIONARY_PS_UPDATED',chunk_size = 500)

	Number of rows loaded: 209
	Number of rows rejected: 0


Total Rows loaded: 209
Total rejected rows: 0


'Successful'

In [ ]:
# missing_terms.to_excel('output/missing_terms_ps_dictionary.xlsx', index=False)

In [131]:
# print_merged = pd.merge(prnt, print_dictionary, how='left', left_on='lemmatized', right_on = 'term')
# relevant_print = print_merged.loc[(~print_merged['type'].isin(['stopwords', 'default'])) | (print_merged['type'].isnull())]
# relevant_print

ps_merged = pd.merge(ps, ps_dictionary, how='left', left_on='lemmatized', right_on = 'term')
relevant_ps = ps_merged.loc[(~ps_merged['type'].isin(['stopwords', 'default'])) | (ps_merged['type'].isnull())]
relevant_ps

,case_id,case_closed_date,ip2,business,platform_name,country_code,geo_market,term_x,score,count_x,word_length,lemmatized,product_category,end_user_company_name,serial_number,product,Segment,Sub_Reg,Region,Unnamed: 0,Unnamed: 0.1,term_y,type,theme,topic,count_y,L2 theme,L1 theme
0,5004214083,2021-11-03T23:50:59.000Z,office_suite,Personal Systems,Bosco 1.0 PAV,US,North America,case number,0.693147,1.0,2,case number,Consumer Notebooks,None,None,None,Consumer,None,None,770.0,2338.0,case number,generic,None,None,60.0,None,None
1,5005341294,2021-11-10T14:56:07.000Z,battery,Personal Systems,Pike 1.1 SPR,US,North America,note type,1.386294,2.0,2,note type,Consumer Notebooks,None,None,None,Consumer,None,None,4233.0,3471.0,note type,generic,None,None,62.0,None,None
2,5005341294,2021-11-10T14:56:07.000Z,battery,Personal Systems,Pike 1.1 SPR,US,North America,problem description,0.693147,1.0,2,problem description,Consumer Notebooks,None,None,None,Consumer,None,None,11033.0,4110.0,problem description,generic,None,None,150.0,None,None
3,5005341294,2021-11-10T14:56:07.000Z,battery,Personal Systems,Pike 1.1 SPR,US,North America,problem description,0.693147,1.0,2,problem description,Consumer Notebooks,None,None,None,Consumer,None,None,NaN,NaN,problem description,None,None,None,15835.0,None,None
4,5005341294,2021-11-10T14:56:07.000Z,battery,Personal Systems,Pike 1.1 SPR,US,North America,problem description,0.693147,1.0,2,problem description,Consumer Notebooks,None,None,None,Consumer,None,None,NaN,NaN,problem description,None,None,None,15835.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1251293,5075754023,2021-11-13T19:20:43.000Z,product_specs_question,Personal Systems,Tortilla 1.0 HPP,US,North America,right one,0.693147,1.0,2,right one,Consumer Notebooks,None,None,None,Consumer,None,None,5270.0,6653.0,right one,generic,None,None,67.0,None,None
1251294,5075754028,2021-11-13T19:23:19.000Z,cosmetic_unit_damage,Personal Systems,Snape 1.1 HPP,US,North America,computer screen,0.693147,1.0,2,computer screen,Consumer Notebooks,None,None,None,Consumer,None,None,3435.0,1079.0,computer screen,generic,None,None,49.0,None,None
1251295,5075754028,2021-11-13T19:23:19.000Z,cosmetic_unit_damage,Personal Systems,Snape 1.1 HPP,US,North America,customer issue,0.693147,1.0,2,customer issue,Consumer Notebooks,None,None,None,Consumer,None,None,417.0,1236.0,customer issue,generic,None,None,42.0,None,None
1251296,5075754075,2021-11-13T19:26:31.000Z,internet_related,Personal Systems,Andaman_19C2,US,North America,connectivity issue,0.693147,1.0,2,connectivity issue,Consumer Notebooks,None,None,None,Consumer,None,None,12710.0,9019.0,connectivity issue,specific,connectivity issue,network issue,NaN,connectivity issue,connectivity


In [133]:
missing_terms_ps = pd.DataFrame(relevant_ps[relevant_ps.type.isnull()].lemmatized.value_counts())
missing_terms_ps.reset_index(inplace=True)
missing_terms_ps.columns = ['lemmatized', 'count']
missing_terms_ps

# missing_terms_print = pd.DataFrame(relevant_print[relevant_print.type.isnull()].lemmatized.value_counts())
# missing_terms_print.reset_index(inplace=True)
# missing_terms_print.columns = ['lemmatized', 'count']
# missing_terms_print

,lemmatized,count
0,problem description,97245
1,order reference number,23781
2,service order,22791
3,onsite service,12600
4,power button,12513
...,...,...
520,printer full software,1
521,license miss,1
522,only color,1
523,country region,1


In [81]:
# hlp.load_to_vertica_database(missing_terms_print, hlp.schema, 'EWS_DICTIONARY_PRINT')
missing_terms_print.to_excel('output/missing_terms_print_dictionary.xlsx', index=False)

NameError: name 'missing_terms_print' is not defined

In [45]:
def get_ridf(df, term_col=None, id_col=None, verbose=False):
    
    if id_col is None:
        id_col = 'case_id'
    
    if term_col is None:
        term_col = 'lemmatized'
    
    terms = df[term_col].unique()
    case_ids = df[id_col].unique()
    total_cases = len(case_ids)
    
    term_group = df[[id_col, term_col, 'count_x']].groupby(term_col).agg({id_col: pd.Series.nunique, 'count_x': np.sum})
    term_group = term_group[term_group[id_col] >= 30]
    
    term_group['df'] = term_group[id_col] / total_cases
    term_group['idf'] = np.log(1 / term_group['df'])
    
    term_group['expected_idf'] = -np.log(1 - np.exp(-term_group['count_x'] / total_cases))
    term_group['ridf'] = term_group['idf'] - term_group['expected_idf']
    
    term_group.reset_index(inplace=True)
    term_group.sort_values(by=['ridf'], ascending=False, inplace=True, ignore_index=True)
    term_group = term_group.head(500)
    
    return term_group

In [46]:
relevant_ps

,case_id,case_closed_date,ip2,business,platform_name,country_code,geo_market,term_x,score,count_x,...,Region,Unnamed: 0,Unnamed: 0.1,term_y,type,theme,topic,count_y,L2 theme,L1 theme
0,5013036008,2021-11-18T02:36:10.000Z,windows_tools_and_applications,Personal Systems,Burrito 1.0 HPP,US,North America,same message,1.386294,2.0,...,None,11148.0,4451.0,same message,generic,None,None,82.0,None,None
1,5013660197,2021-11-15T09:36:54.000Z,warranty_entitlement,Personal Systems,TexansT 1.0,US,North America,same case,0.693147,1.0,...,None,1451.0,4434.0,same case,generic,None,None,2.0,None,None
2,5013983689,2021-11-19T08:31:27.000Z,battery,Personal Systems,Jaguars 1.0 HPP,NL,Northern Europe,error message,0.693147,1.0,...,None,3608.0,1599.0,error message,generic,None,None,38.0,None,None
3,5019551471,2021-11-15T11:42:18.000Z,windows_tools_and_applications,Personal Systems,17 2C HPAIO ProstT Win10-64,US,North America,customer issue,0.693147,1.0,...,None,417.0,1236.0,customer issue,generic,None,None,42.0,None,None
4,5019551471,2021-11-15T11:42:18.000Z,windows_tools_and_applications,Personal Systems,17 2C HPAIO ProstT Win10-64,US,North America,device manager,0.693147,1.0,...,None,3531.0,1374.0,device manager,specific,device manager,None,4960.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803500,5076117439,2021-11-19T19:27:15.000Z,sleep_hibernate,Personal Systems,Maldives_20C1,US,North America,previous case,1.443147,1.0,...,None,NaN,NaN,previous case,None,None,None,891.0,None,None
803501,5076117439,2021-11-19T19:27:15.000Z,sleep_hibernate,Personal Systems,Maldives_20C1,US,North America,previous case,1.443147,1.0,...,None,NaN,NaN,previous case,None,None,None,7719.0,None,None
803502,5076117574,2021-11-19T19:13:11.000Z,drivers_and_updates,Personal Systems,Carp 1.0 SPR,US,North America,drivers issue,0.693147,1.0,...,None,10283.0,1910.0,driver issue,specific,driver related issue,None,33.0,driver issue,firmware/software/driver
803503,5076117687,2021-11-19T19:24:37.000Z,hinges,Personal Systems,Snape_20C1,US,North America,repair option,1.386294,2.0,...,None,4520.0,4309.0,repair option,generic,None,None,112.0,None,None


In [47]:
ps_ridf = get_ridf(relevant_ps)
# prnt_ridf = get_ridf(relevant_print)

In [48]:
ps_ridf
#prnt_ridf

,lemmatized,case_id,count_x,df,idf,expected_idf,ridf
0,n a,70,2560.0,0.001089,6.822711,3.243286,3.579425
1,keyboard issue,1577,61434.0,0.024528,3.707926,0.485495,3.222431
2,mouse issue,213,4473.0,0.003313,5.709914,2.699976,3.009938
3,webcam issue,252,3605.0,0.003920,5.541777,2.909034,2.632743
4,power button,2298,33378.0,0.035743,3.331412,0.903926,2.427486
...,...,...,...,...,...,...,...
495,bios page,270,405.0,0.004200,5.472784,5.070467,0.402317
496,password issue,179,268.0,0.002784,5.883820,5.482303,0.401518
497,live lens,796,1200.0,0.012381,4.391607,3.990447,0.401160
498,usb stick,65,97.0,0.001011,6.896819,6.497249,0.399569


In [49]:
pd.set_option('display.max_rows', 100)
ps_ridf.head(100)

,lemmatized,case_id,count_x,df,idf,expected_idf,ridf
0,n a,70,2560.0,0.001089,6.822711,3.243286,3.579425
1,keyboard issue,1577,61434.0,0.024528,3.707926,0.485495,3.222431
2,mouse issue,213,4473.0,0.003313,5.709914,2.699976,3.009938
3,webcam issue,252,3605.0,0.003920,5.541777,2.909034,2.632743
4,power button,2298,33378.0,0.035743,3.331412,0.903926,2.427486
5,kind regard,576,6675.0,0.008959,4.715098,2.316543,2.398555
6,well regard,663,7495.0,0.010312,4.574431,2.206936,2.367495
7,touchpad issue,477,5110.0,0.007419,4.903690,2.571728,2.331961
8,hard drive,2126,26060.0,0.033067,3.409208,1.098879,2.310330
9,care pack,292,2985.0,0.004542,5.394452,3.092975,2.301477


In [51]:
refresh_table_and_backups(ps_ridf, f'EWS_RIDF_PS_{segment}')
#refresh_table_and_backups(prnt_ridf, f'EWS_RIDF_PRINT_{segment}')

Running 
drop table if exists pps_quality.EWS_RIDF_PS_CONSUMER;
 on aws3
Ran successfully!
Running 
alter table pps_quality.EWS_RIDF_PS_CONSUMER_PREV_1
                                rename to EWS_RIDF_PS_CONSUMER_PREV_2;
 on aws3
Failed!
Due to Severity: ROLLBACK, Message: A table named "EWS_RIDF_PS_CONSUMER_PREV_2" already exists, Sqlstate: 42710, Routine: validateRelationNameDoesNotExist, File: /scratch_a/release/svrtar18593/vbuild/vertica/Commands/DDL.cpp, Line: 20949, Error Code: 2105, SQL: 'alter table pps_quality.EWS_RIDF_PS_CONSUMER_PREV_1                                 rename to EWS_RIDF_PS_CONSUMER_PREV_2;'
Running 
alter table pps_quality.EWS_RIDF_PS_CONSUMER
                                rename to EWS_RIDF_PS_CONSUMER_PREV_1;
 on aws3
Failed!
Due to Severity: ROLLBACK, Message: Table "EWS_RIDF_PS_CONSUMER" does not exist, Sqlstate: 42704, Routine: RenameTable, File: /scratch_a/release/svrtar18593/vbuild/vertica/Commands/DDL.cpp, Line: 1800, Error Code: 4876, SQL: 'alter

In [52]:
q = f"""select * from {hlp.schema}.EWS_DICTIONARY_PS_UPDATED"""
with hlp.get_connection() as aws_conn:
    ps_dict_df = pd.read_sql(q, aws_conn)
# ps_dict_df.drop(columns = ['Unnamed: 7','Unnamed: 0.1','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11'],inplace = True)
ps_dict_df

,Unnamed: 0,Unnamed: 0.1,term,type,theme,topic,count,L2 theme,L1 theme
0,0.0,0.0,ac adpater,specific,ac power cord,None,48.0,ac adaptor,power
1,2.0,3.0,audio jack issue,specific,audio issue,None,43.0,audio issue,button
2,3.0,8.0,battery calibration,specific,calibration issue,battery issue,NaN,battery calibration,battery
3,8.0,28.0,computer freeze,specific,computer freeze,None,162.0,computer freeze,firmware/software/driver
4,10.0,32.0,damage device,generic,damage device,damage,NaN,damage device,other
...,...,...,...,...,...,...,...,...,...
24326,NaN,NaN,valid case,None,None,None,872.0,None,None
24327,NaN,NaN,valid serial number,None,None,None,644.0,None,None
24328,NaN,NaN,wall socket,None,None,None,1528.0,None,None
24329,NaN,NaN,work order schedule,None,None,None,57.0,None,None


In [85]:
high_lev_themes = pd.read_excel('Copy of High level themes.xlsx')
ps_dict_df_merged = ps_dict_df.merge(high_lev_themes, how = 'left',left_on = 'theme', right_on = 'L2 theme') 
ps_dict_df_merged

,Unnamed: 0,term,type,theme,topic,count,L2 theme_x,L1 theme_x,L2 theme_y,L1 theme_y
0,4.0,bad fan,specific,bad fan,None,61.0,bad fan,thermal,bad fan,thermal
1,5.0,bad hdd,specific,hard disk drive issue,hard drive issue,NaN,hard disk drive issue,storage,hard disk drive issue,storage
2,9.0,battery drain issue,specific,battery drain issue,None,38.0,battery drain issue,battery,battery drain issue,battery
3,11.0,bb swell battery,specific,swollen battery,None,62.0,swollen battery,battery,swollen battery,battery
4,14.0,bios mode f10,specific,bios mode f10,None,117.0,bios mode f10,boot,bios mode f10,boot
...,...,...,...,...,...,...,...,...,...,...
19817,NaN,user invoice date,None,None,None,50.0,None,None,NaN,NaN
19818,NaN,whole printer,None,None,None,1.0,None,None,NaN,NaN
19819,NaN,wide format,None,None,None,1.0,None,None,NaN,NaN
19820,NaN,window defender,None,None,None,33.0,None,None,NaN,NaN


In [86]:
print(len(df))
df = df.drop_duplicates()
len(df)

NameError: name 'df' is not defined

In [ ]:
drop_q = """drop table pps_quality.EWS_DICTIONARY_PS_UPDATED"""
hlp.run_query(drop_q)

In [ ]:
hlp.load_to_vertica_database(ps_dict_df_merged,hlp.schema,'EWS_DICTIONARY_PS_UPDATED')

In [104]:
# q = """select copy_table('pps_quality.EWS_RIDF_PS_COMMERCIAL_PREV_1', 'pps_quality.EWS_RIDF_PS_COMMERCIAL')"""
# hlp.run_query(q)

In [105]:
# adhoc_refresh(ps_ridf, 'EWS_RIDF_PS_CONSUMER')

In [106]:
# refresh_table_and_backups(ps_ridf, 'EWS_RIDF_PS')
# refresh_table_and_backups(prnt_ridf, 'EWS_RIDF_PRINT')

### TFIDF refresh for terms - PS

In [87]:
def get_tables(business, segment, iteration_suffix='current'):
    selection_string = f'{business}_{segment}'
    
    if iteration_suffix != 'current':
        selection_string += f'_{iteration_suffix}'
    
    tables = {
        'tfidf': f'EWS_TFIDF_{selection_string}',
        'ridf': f'EWS_RIDF_{selection_string}',
        'master': f'EWS_MASTER_{selection_string}',
    }
    
    return tables

In [88]:
def run_tfidf_pipeline(business, segment, 
                       iteration_suffix='current', 
                       threshold = 30, 
                       id_col = 'case_id', 
                       grouping_frequency = "1W", 
                       load_to_database=False,
                       verbose=False):
    tables = get_tables(business=business, segment=segment, iteration_suffix=iteration_suffix)
    query = f"""select distinct lemmatized from pps_quality.{tables['ridf']} where ridf > 0;"""
    
    ridf, _ = hlp.get_data(query)
    ridf_terms = ridf.lemmatized.to_list()
    ridf_terms[:5]
    ridf_terms_str = "', '".join(ridf_terms)
    
    query = f"""select case_id, 
            date({date_col}) as case_closed_date,
            business, lemmatized
            from {hlp.schema}.{restructured_table}
            where date(case_closed_date) >= '{start_date}'
            and business = '{'Personal Systems' if business=='PS' else 'Printing'}'
            and lower(segment) = '{segment.lower()}'
            and lemmatized in (
                '{ridf_terms_str}'
            );"""
    
    with hlp.get_connection() as aws_conn:
        restructured_df = pd.read_sql(query, aws_conn, parse_dates=[date_col])\
        
    filtered = restructured_df.loc[restructured_df.lemmatized.map(restructured_df.lemmatized.value_counts()) >= threshold, :]
    
    if verbose:
        print(f'Total terms: {len(restructured_df)}')
        print(f'Total terms after applying threshold: {len(filtered)}')
        
    # Getting week wise counts
    
    grouped = filtered[[id_col, date_col, 'lemmatized']].groupby(['lemmatized', 
                                                                    pd.Grouper(key=date_col, 
                                                                               freq=grouping_frequency)
                                                                    ]).nunique().unstack()
    grouped.fillna(0, inplace=True)
    total_cases = len(filtered.case_id.unique())
    grouped = grouped.droplevel([0], axis=1)
    date_cols = [col for col in grouped.columns if col != 'lemmatized']
    
    temp = grouped.copy()
    
    col_rename_map = {}
    for col in date_cols:
        col_rename_map.update({col: col.strftime('%y-%W')})
    
    temp = temp.rename(col_rename_map, axis=1)
    
    for col in temp.columns:
        if col == 'lemmatized':
            continue

        temp[f'{col}_tf'] = temp[col] / temp[col].sum()  #TF
        temp[f'{col}_df'] = temp[col_rename_map.values()].sum(axis=1) / total_cases  #DF
        temp[f'{col}_idf'] = np.log(1 / temp[f'{col}_df'])
        temp[f'{col}_tfidf'] = temp[f'{col}_tf'] * temp[f'{col}_idf']

        temp[f'{col}_tf_log'] = np.log(1 + temp[f'{col}_tf'])
        temp[f'{col}_df_log'] = np.log(1 + temp[f'{col}_df'])
        temp[f'{col}_idf_log'] = np.log(1 / temp[f'{col}_df_log'])
        temp[f'{col}_tfidf_log'] = temp[f'{col}_tf_log'] * temp[f'{col}_idf_log']
    
    
    temp.reset_index(inplace=True)    
    tfidf_suffixes = ['tf', 'df', 'idf', 'tfidf', 'tf_log', 'df_log', 'idf_log', 'tfidf_log']
    
    date_cols = list(col_rename_map.values())
    tf_cols = [f'{col}_tf' for col in date_cols]
    df_cols = [f'{col}_df' for col in date_cols]
    idf_cols = [f'{col}_idf' for col in date_cols]
    tfidf_cols = [f'{col}_tfidf' for col in date_cols]
    tf_log_cols = [f'{col}_tf_log' for col in date_cols]
    df_log_cols = [f'{col}_df_log' for col in date_cols]
    tfidf_log_cols = [f'{col}_tfidf_log' for col in date_cols]
    
    counts_df = pd.melt(temp, id_vars=['lemmatized'], value_vars=date_cols)
    tf_df = pd.melt(temp, id_vars=['lemmatized'], value_vars=tf_cols)
    df_df = pd.melt(temp, id_vars=['lemmatized'], value_vars=df_cols)
    idf_df = pd.melt(temp, id_vars=['lemmatized'], value_vars=idf_cols)
    tfidf_df = pd.melt(temp, id_vars=['lemmatized'], value_vars=tfidf_cols)
    tf_log_df = pd.melt(temp, id_vars=['lemmatized'], value_vars=tf_log_cols)
    df_log_df = pd.melt(temp, id_vars=['lemmatized'], value_vars=df_log_cols)
    tfidf_log_df = pd.melt(temp, id_vars=['lemmatized'], value_vars=tfidf_log_cols)
    
    tfidf_df.case_closed_date = tfidf_df.case_closed_date.str.replace('_tfidf', '')
    tf_df.case_closed_date = tf_df.case_closed_date.str.replace('_tf', '')
    df_df.case_closed_date = df_df.case_closed_date.str.replace('_df', '')
    tfidf_log_df.case_closed_date = tfidf_log_df.case_closed_date.str.replace('_tfidf_log', '')
    tf_log_df.case_closed_date = tf_log_df.case_closed_date.str.replace('_tf_log', '')
    df_log_df.case_closed_date = df_log_df.case_closed_date.str.replace('_df_log', '')

    counts_df = counts_df.rename({'value': 'count'}, axis=1)
    tfidf_df = tfidf_df.rename({'value': 'tfidf'}, axis=1)
    tf_df = tf_df.rename({'value': 'tf'}, axis=1)
    df_df = df_df.rename({'value': 'df'}, axis=1)
    tfidf_log_df = tfidf_log_df.rename({'value': 'tfidf_log'}, axis=1)
    tf_log_df = tf_log_df.rename({'value': 'tf_log'}, axis=1)
    df_log_df = df_log_df.rename({'value': 'df_log'}, axis=1)

    c_t = pd.merge(left=counts_df,
                  right=tf_df,
                  how='left',
                  on=['lemmatized', 'case_closed_date'],
                  suffixes = ('_count', '_tf'))

    c_t_d = pd.merge(left=c_t,
                  right=df_df,
                  how='left',
                  on=['lemmatized', 'case_closed_date'],
                  suffixes = ('_count', '_df'))

    c_t_d_tf = pd.merge(left=c_t_d,
                  right=tfidf_df,
                  how='left',
                  on=['lemmatized', 'case_closed_date'],
                  suffixes = ('_count', '_tfidf'))

    c_t_d_tf = pd.merge(left=c_t_d_tf,
                  right=df_log_df,
                  how='left',
                  on=['lemmatized', 'case_closed_date'],
                  suffixes = ('_count', '_df_log'))
    c_t_d_tf = pd.merge(left=c_t_d_tf,
                  right=tf_log_df,
                  how='left',
                  on=['lemmatized', 'case_closed_date'],
                  suffixes = ('_count', '_tf_log'))
    c_t_d_tf = pd.merge(left=c_t_d_tf,
                  right=tfidf_log_df,
                  how='left',
                  on=['lemmatized', 'case_closed_date'],
                  suffixes = ('_count', '_tfidf_log'))

    final = c_t_d_tf
    final = final.sort_values(by=['lemmatized', 'case_closed_date'], ignore_index=True)
    if load_to_database:
        refresh_table_and_backups(final, tables['tfidf'])
    
    
    return final

In [89]:
businesses = ['PRINT', 'PS']
segments = ['CONSUMER', 'COMMERCIAL']

for business in businesses:
    for segment in segments:
        print(business, segment)
        temp = run_tfidf_pipeline(business, segment, load_to_database=True)
        display(temp)
# temp = run_tfidf_pipeline('PRINT', 'COMMERCIAL')
# temp

PRINT CONSUMER
Final record_count: 500
Running 
drop table if exists pps_quality.EWS_TFIDF_PRINT_CONSUMER;
 on aws3
Ran successfully!
Running 
alter table pps_quality.EWS_TFIDF_PRINT_CONSUMER_PREV_1
                                rename to EWS_TFIDF_PRINT_CONSUMER_PREV_2;
 on aws3
Failed!
Due to Severity: ROLLBACK, Message: A table named "EWS_TFIDF_PRINT_CONSUMER_PREV_2" already exists, Sqlstate: 42710, Routine: validateRelationNameDoesNotExist, File: /scratch_a/release/svrtar18593/vbuild/vertica/Commands/DDL.cpp, Line: 20949, Error Code: 2105, SQL: 'alter table pps_quality.EWS_TFIDF_PRINT_CONSUMER_PREV_1                                 rename to EWS_TFIDF_PRINT_CONSUMER_PREV_2;'
Running 
alter table pps_quality.EWS_TFIDF_PRINT_CONSUMER
                                rename to EWS_TFIDF_PRINT_CONSUMER_PREV_1;
 on aws3
Failed!
Due to Severity: ROLLBACK, Message: Table "EWS_TFIDF_PRINT_CONSUMER" does not exist, Sqlstate: 42704, Routine: RenameTable, File: /scratch_a/release/svrtar18593

,lemmatized,case_closed_date,count,tf,df,tfidf,df_log,tf_log,tfidf_log
0,-no issue,20-39,0.0,0.000000,0.000178,0.000000,0.000178,0.000000,0.000000
1,-no issue,20-40,0.0,0.000000,0.000178,0.000000,0.000178,0.000000,0.000000
2,-no issue,20-41,0.0,0.000000,0.000178,0.000000,0.000178,0.000000,0.000000
3,-no issue,20-42,0.0,0.000000,0.000178,0.000000,0.000178,0.000000,0.000000
4,-no issue,20-43,0.0,0.000000,0.000178,0.000000,0.000178,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
25839,yellow original ink,21-34,13.0,0.000198,0.001362,0.001306,0.001362,0.000198,0.001306
25840,yellow original ink,21-35,19.0,0.000301,0.001362,0.001984,0.001362,0.000301,0.001984
25841,yellow original ink,21-36,14.0,0.000241,0.001362,0.001588,0.001362,0.000241,0.001588
25842,yellow original ink,21-37,22.0,0.000351,0.001362,0.002319,0.001362,0.000351,0.002319


PRINT COMMERCIAL
Final record_count: 478
Running 
drop table if exists pps_quality.EWS_TFIDF_PRINT_COMMERCIAL;
 on aws3
Ran successfully!
Running 
alter table pps_quality.EWS_TFIDF_PRINT_COMMERCIAL_PREV_1
                                rename to EWS_TFIDF_PRINT_COMMERCIAL_PREV_2;
 on aws3
Failed!
Due to Severity: ROLLBACK, Message: A table named "EWS_TFIDF_PRINT_COMMERCIAL_PREV_2" already exists, Sqlstate: 42710, Routine: validateRelationNameDoesNotExist, File: /scratch_a/release/svrtar18593/vbuild/vertica/Commands/DDL.cpp, Line: 20949, Error Code: 2105, SQL: 'alter table pps_quality.EWS_TFIDF_PRINT_COMMERCIAL_PREV_1                                 rename to EWS_TFIDF_PRINT_COMMERCIAL_PREV_2;'
Running 
alter table pps_quality.EWS_TFIDF_PRINT_COMMERCIAL
                                rename to EWS_TFIDF_PRINT_COMMERCIAL_PREV_1;
 on aws3
Failed!
Due to Severity: ROLLBACK, Message: Table "EWS_TFIDF_PRINT_COMMERCIAL" does not exist, Sqlstate: 42704, Routine: RenameTable, File: /scratch_a

,lemmatized,case_closed_date,count,tf,df,tfidf,df_log,tf_log,tfidf_log
0,110v fuser,20-39,18.0,0.001423,0.005157,0.007495,0.005144,0.001422,0.007493
1,110v fuser,20-40,46.0,0.001683,0.005157,0.008864,0.005144,0.001681,0.008861
2,110v fuser,20-41,52.0,0.001710,0.005157,0.009007,0.005144,0.001708,0.009004
3,110v fuser,20-42,43.0,0.001566,0.005157,0.008247,0.005144,0.001564,0.008244
4,110v fuser,20-43,43.0,0.001583,0.005157,0.008341,0.005144,0.001582,0.008338
...,...,...,...,...,...,...,...,...,...
24695,﻿1 interaction,21-34,20.0,0.000545,0.000874,0.003838,0.000874,0.000545,0.003837
24696,﻿1 interaction,21-35,15.0,0.000418,0.000874,0.002943,0.000874,0.000418,0.002943
24697,﻿1 interaction,21-36,12.0,0.000364,0.000874,0.002566,0.000874,0.000364,0.002566
24698,﻿1 interaction,21-37,12.0,0.000330,0.000874,0.002327,0.000874,0.000330,0.002327


PS CONSUMER
Final record_count: 500
Running 
drop table if exists pps_quality.EWS_TFIDF_PS_CONSUMER;
 on aws3
Ran successfully!
Running 
alter table pps_quality.EWS_TFIDF_PS_CONSUMER_PREV_1
                                rename to EWS_TFIDF_PS_CONSUMER_PREV_2;
 on aws3
Failed!
Due to Severity: ROLLBACK, Message: A table named "EWS_TFIDF_PS_CONSUMER_PREV_2" already exists, Sqlstate: 42710, Routine: validateRelationNameDoesNotExist, File: /scratch_a/release/svrtar18593/vbuild/vertica/Commands/DDL.cpp, Line: 20949, Error Code: 2105, SQL: 'alter table pps_quality.EWS_TFIDF_PS_CONSUMER_PREV_1                                 rename to EWS_TFIDF_PS_CONSUMER_PREV_2;'
Running 
alter table pps_quality.EWS_TFIDF_PS_CONSUMER
                                rename to EWS_TFIDF_PS_CONSUMER_PREV_1;
 on aws3
Failed!
Due to Severity: ROLLBACK, Message: Table "EWS_TFIDF_PS_CONSUMER" does not exist, Sqlstate: 42704, Routine: RenameTable, File: /scratch_a/release/svrtar18593/vbuild/vertica/Commands/DDL.c

,lemmatized,case_closed_date,count,tf,df,tfidf,df_log,tf_log,tfidf_log
0,-no go,20-39,17.0,0.001159,0.001177,0.007816,0.001177,0.001158,0.007812
1,-no go,20-40,29.0,0.000823,0.001177,0.005552,0.001177,0.000823,0.005550
2,-no go,20-41,25.0,0.000728,0.001177,0.004907,0.001177,0.000727,0.004906
3,-no go,20-42,16.0,0.000471,0.001177,0.003177,0.001177,0.000471,0.003176
4,-no go,20-43,22.0,0.000633,0.001177,0.004272,0.001177,0.000633,0.004271
...,...,...,...,...,...,...,...,...,...
25839,﻿1 floor,21-34,8.0,0.000191,0.000531,0.001442,0.000531,0.000191,0.001442
25840,﻿1 floor,21-35,22.0,0.000500,0.000531,0.003768,0.000531,0.000500,0.003767
25841,﻿1 floor,21-36,12.0,0.000319,0.000531,0.002406,0.000531,0.000319,0.002405
25842,﻿1 floor,21-37,18.0,0.000433,0.000531,0.003269,0.000531,0.000433,0.003268


PS COMMERCIAL
Final record_count: 500
Running 
drop table if exists pps_quality.EWS_TFIDF_PS_COMMERCIAL;
 on aws3
Ran successfully!
Running 
alter table pps_quality.EWS_TFIDF_PS_COMMERCIAL_PREV_1
                                rename to EWS_TFIDF_PS_COMMERCIAL_PREV_2;
 on aws3
Failed!
Due to Severity: ROLLBACK, Message: A table named "EWS_TFIDF_PS_COMMERCIAL_PREV_2" already exists, Sqlstate: 42710, Routine: validateRelationNameDoesNotExist, File: /scratch_a/release/svrtar18593/vbuild/vertica/Commands/DDL.cpp, Line: 20949, Error Code: 2105, SQL: 'alter table pps_quality.EWS_TFIDF_PS_COMMERCIAL_PREV_1                                 rename to EWS_TFIDF_PS_COMMERCIAL_PREV_2;'
Running 
alter table pps_quality.EWS_TFIDF_PS_COMMERCIAL
                                rename to EWS_TFIDF_PS_COMMERCIAL_PREV_1;
 on aws3
Failed!
Due to Severity: ROLLBACK, Message: Table "EWS_TFIDF_PS_COMMERCIAL" does not exist, Sqlstate: 42704, Routine: RenameTable, File: /scratch_a/release/svrtar18593/vbuild/ve

,lemmatized,case_closed_date,count,tf,df,tfidf,df_log,tf_log,tfidf_log
0,.informed customer,20-39,0.0,0.000000,0.000249,0.000000,0.000249,0.000000,0.000000
1,.informed customer,20-40,0.0,0.000000,0.000249,0.000000,0.000249,0.000000,0.000000
2,.informed customer,20-41,1.0,0.000020,0.000249,0.000167,0.000249,0.000020,0.000167
3,.informed customer,20-42,0.0,0.000000,0.000249,0.000000,0.000249,0.000000,0.000000
4,.informed customer,20-43,0.0,0.000000,0.000249,0.000000,0.000249,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
25943,﻿1 floor,21-34,19.0,0.000348,0.000542,0.002618,0.000542,0.000348,0.002617
25944,﻿1 floor,21-35,28.0,0.000536,0.000542,0.004028,0.000542,0.000535,0.004027
25945,﻿1 floor,21-36,14.0,0.000259,0.000542,0.001946,0.000542,0.000259,0.001946
25946,﻿1 floor,21-37,23.0,0.000383,0.000542,0.002878,0.000542,0.000383,0.002878


### Identify Weak Signals (Refactored)

All the rules are based in the TF-IDF value: 
- TFIDF the higher the more relevant the term in that period
- WoW when to continues TF IDF are higher tan the previous- term is getting more revelant
- Control chart – term is outside of normal relevance for it’s behaviour
- Average higher twice – term relevance is continuosly higher than the norm
- Higher lows – term relevance has not drop enough to not be significant. 
- RIDF – the terms get moduated into not too high and not too low. – the higher the more in between terms. 


In [91]:
get_tables('PS', 'COMMERCIAL')

{'tfidf': 'EWS_TFIDF_PS_COMMERCIAL',
 'ridf': 'EWS_RIDF_PS_COMMERCIAL',
 'master': 'EWS_MASTER_PS_COMMERCIAL'}

In [92]:
master_table = 'EWS_PYATE_NOTES_RESTRUCTURED'
term_col = 'lemmatized'
id_col = 'case_id'
date_col = 'case_closed_date'
sub_col = 'case_subject_cleaned'

def get_weak_signals(business, segment, iteration_suffix='current', weeks=1, WoW_weeks = 3, 
                     load_to_database=False, adhoc_refresh_flag=False, verbose=True, col='tf'):
    
    # Get table names for the given business, segment and iteration
    tables = get_tables(business=business, 
                        segment=segment, 
                        iteration_suffix=iteration_suffix
                       )
    
    if verbose:
        print(f'TFIDF table: {tables["tfidf"]}')
        print(f'RIDF table: {tables["ridf"]}')
        print(f'Master table: {tables["master"]}')
    
    # get tfidf data
    tfidf, _ = hlp.get_data(f"""select lemmatized, case_closed_date, count, tfidf, tf from {hlp.schema}.{tables["tfidf"]}""")
    tfidf.drop_duplicates(inplace=True)
    
    # convert case_closed_date format
    tfidf[['Year','Week']] = tfidf.case_closed_date.str.split("-",expand=True,)
    dates = '20'+tfidf.Year+tfidf.Week
    
    tfidf['case_closed_date'] = pd.to_datetime(dates.astype(str) + '0', format='%Y%W%w')
    tfidf = tfidf.drop(['Year','Week'], axis=1)
    
    # Set processing dates
    last_mon = max(tfidf['case_closed_date'])  - pd.DateOffset(weeks=weeks)
    prev_mon = last_mon - pd.DateOffset(weeks=1)
    prev_mon_2 = last_mon - pd.DateOffset(weeks=2)
    
#     if verbose:
#         print(f'\nLast Monday: \t{last_mon}')
#         print(f'Previous Monday: \t{prev_mon}')
#         print(f'Previous Monday 2: \t{prev_mon_2}')
    
    # create results dataframe structure and populate terms col
    weak_sig = pd.DataFrame(columns=['Terms', 'WoW', 'CoCh', 'MAvg', 'HiLo', 'delta'])
    weak_sig['Terms'] = tfidf['lemmatized'].unique()
    
#     print(tfidf)
#     tfidf =  populate_percent_contribution(tfidf,business,segment)
    

    # Get WoW
    weak_sig = populate_WoW_flag(tfidf, weak_sig, weeks=WoW_weeks, verbose=verbose, col=col)
    
    # Get CoCh and MAvg
    weak_sig = populate_bollinger_band_flags(tfidf, weak_sig, window_size=4,
                                 last_mon = last_mon, prev_mon = prev_mon, prev_mon_2 = prev_mon_2, verbose=verbose, col=col)
    
    # Get HiLo
    weak_sig = populate_HiLo_Flag(tfidf, weak_sig,
                                 last_mon = last_mon, prev_mon = prev_mon, prev_mon_2 = prev_mon_2, verbose=verbose, col=col)
    
    # Populate last tfidf and count cols
    for term in tfidf['lemmatized'].unique():
        tfidf_last = tfidf[(tfidf['lemmatized']==term) & (tfidf['case_closed_date']==last_mon)]['tfidf'].item()
        count_last = tfidf[(tfidf['lemmatized']==term) & (tfidf['case_closed_date']==last_mon)]['count'].item()
        weak_sig.loc[weak_sig['Terms']==term,'count'] = count_last
        weak_sig.loc[weak_sig['Terms']==term,'tfidf'] = tfidf_last
    
    
    # Compute total
    weak_sig = weak_sig.fillna(0)
    weak_sig['total'] = weak_sig['WoW'] + weak_sig['CoCh'] + weak_sig['MAvg'] + weak_sig['HiLo'] + 1
    
    # get ridf data
    ridf, _ = hlp.get_data(f"""select * from {hlp.schema}.{tables["ridf"]};""")
    
    
    tfidf_merged_with_ridf = pd.merge(how='left',left=weak_sig, right=ridf, left_on='Terms', right_on='lemmatized')
    tfidf_merged_with_ridf.fillna(0, inplace=True)
    
#     if verbose:
#         display(tfidf_merged_with_ridf.head())
    
    # scale columns before multiplication
    df = generate_scores(tfidf_merged_with_ridf, col=col)
    
    final_output = df[(df['count'] > 50)].sort_values(by=['we_sig_ridf'], ascending=False, ignore_index=True)
    
    if verbose:
        pd.set_option('display.max_rows', 150)
        display(final_output)
        
    if load_to_database:
        refresh_table_and_backups(final_output, tables["master"])
        
    if adhoc_refresh_flag:
        adhoc_refresh(final_output, tables["master"])
    
    return final_output    

In [93]:
def populate_WoW_flag(tfidf, weak_sig, weeks=3, verbose=True, col='tf'):
    dates = tfidf['case_closed_date'].unique()
    dates = sorted(dates, reverse=True)
        
    for term in tfidf['lemmatized'].unique():
        df_inter = tfidf[tfidf['lemmatized']==term]
        weekly = df_inter[df_inter['case_closed_date'].isin(dates[:weeks])] 
        weekly = weekly.sort_values(by=['case_closed_date'], ascending=False)
        
        if weekly[col].is_monotonic_decreasing:
            weak_sig.loc[weak_sig['Terms']==term,'WoW'] = 1
        else:
            weak_sig.loc[weak_sig['Terms']==term,'WoW'] = 0
    
    if verbose:
        print('\nDistribution of WoW flag')
        display(weak_sig['WoW'].value_counts())
    
    return weak_sig

In [94]:
def populate_bollinger_band_flags(tfidf, weak_sig, 
                       last_mon, prev_mon, prev_mon_2, 
                       window_size=12, bollinger_factor = 1, verbose=True, col='tf'):

    for term in tfidf['lemmatized'].unique():
        #creation of the subsets for each term
        sub_df = tfidf[tfidf['lemmatized']==term].sort_values(by=['case_closed_date'])
        sub_df = sub_df.drop(columns='count')

        #creation of the Bollinger bands
        sub_df['m_avg'] = sub_df[col].rolling(window=window_size).mean()
        sub_df['m_std'] = sub_df[col].rolling(window=window_size).std()
        sub_df['m_upper'] = sub_df.apply(lambda row: row.m_avg + (bollinger_factor * row.m_std), axis=1)
        sub_df['m_lower'] = sub_df.apply(lambda row: row.m_avg - (bollinger_factor * row.m_std), axis=1)
        
#         print(f'm_avg')
#         display(sub_df[["lemmatized", "m_avg", "m_upper"]])
        #identification tf_idf higher than upper band
        tfidf_last = sub_df[sub_df['case_closed_date']==last_mon][col].item()
        upper_last = sub_df[sub_df['case_closed_date']==last_mon]['m_upper'].item()
        if tfidf_last > upper_last: 
            weak_sig.loc[weak_sig['Terms']==term,'CoCh'] = 1
            #print(sub_df[sub_df['case_closed_date']==last_mon])

        #identification of two consecutive tf_idf higher than moving average
        tfidf_last = sub_df[sub_df['case_closed_date']==last_mon][col].item()
        avg_last = sub_df[sub_df['case_closed_date']==last_mon]['m_avg'].item()
        tfidf_prev = sub_df[sub_df['case_closed_date']==prev_mon][col].item()
        avg_prev = sub_df[sub_df['case_closed_date']==prev_mon]['m_avg'].item()
        if tfidf_last > avg_last and tfidf_prev > avg_prev:
            weak_sig.loc[weak_sig['Terms']==term,'MAvg'] = 1
            #print(sub_df[sub_df['case_closed_date']==prev_mon])
            #print(sub_df[sub_df['case_closed_date']==last_mon])
    return weak_sig

In [95]:
def populate_HiLo_Flag(tfidf, weak_sig,
                       last_mon, prev_mon, prev_mon_2, verbose=True, col='tf'):
    for i in tfidf['lemmatized'].unique():
        tfidf_last = tfidf[(tfidf['lemmatized']==i) & (tfidf['case_closed_date']==last_mon)][col].item()
        tfidf_prev = tfidf[(tfidf['lemmatized']==i) & (tfidf['case_closed_date']==prev_mon)][col].item()
        tfidf_prev_2 = tfidf[(tfidf['lemmatized']==i) & (tfidf['case_closed_date']==prev_mon_2)][col].item()
        if (tfidf_last<tfidf_prev) & (tfidf_prev>tfidf_prev_2) & (tfidf_last>tfidf_prev_2):
            weak_sig.loc[weak_sig['Terms']==i,'HiLo'] = 1
    
    return weak_sig

In [96]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

def generate_scores(tfidf_merged_with_ridf, scaler = scaler, cols_to_scale=None, col='tf'):
    if cols_to_scale is None:
        cols_to_scale = ['WoW', 'CoCh', 'MAvg', 'HiLo', 'tfidf','ridf', 
#                          'tf'
                        ]
    
    x = tfidf_merged_with_ridf[cols_to_scale].values
    x_scaled = scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled, columns=cols_to_scale)

    df['total'] = tfidf_merged_with_ridf['total']
    df['count'] = tfidf_merged_with_ridf['count_x']
    df.head()
    df['terms'] = tfidf_merged_with_ridf['Terms']
    df['delta'] = tfidf_merged_with_ridf['delta']
    
    df['we_sig_score'] = df['total'] * df[col]
    df['we_sig_ridf'] = df['total'] * df[col] * df['ridf']
    df['we_sig_delta_ridf'] = df['total'] * df['delta'] * df['ridf']
    
    return df

In [97]:
businesses = [
    'PS',
              'PRINT'
]
segments = [
            'CONSUMER', 
            'COMMERCIAL'
]

for business in businesses:
    for segment in segments:
        res = get_weak_signals(business=business, segment=segment, weeks=1, WoW_weeks=4, 
                               load_to_database=True, 
#                                adhoc_refresh_flag=True,
                               col='tfidf',
                               verbose=True)
        display(res)

TFIDF table: EWS_TFIDF_PS_CONSUMER
RIDF table: EWS_RIDF_PS_CONSUMER
Master table: EWS_MASTER_PS_CONSUMER
Final record_count: 25844

Distribution of WoW flag


0    481
1     16
Name: WoW, dtype: int64

Final record_count: 500


,WoW,CoCh,MAvg,HiLo,tfidf,ridf,total,count,terms,delta,we_sig_score,we_sig_ridf,we_sig_delta_ridf
0,0.0,0.0,0.0,0.0,1.000000,0.339980,1,67373.0,same issue,0,1.000000,0.339980,0.0
1,0.0,1.0,1.0,0.0,0.477282,0.229461,3,8060.0,battery issue,0,1.431847,0.328552,0.0
2,0.0,0.0,1.0,1.0,0.537312,0.176716,3,12673.0,power issue,0,1.611937,0.284855,0.0
3,0.0,0.0,1.0,1.0,0.115721,0.725110,3,1933.0,service tax,0,0.347163,0.251731,0.0
4,0.0,1.0,0.0,0.0,0.129869,0.783669,2,2140.0,material availability,0,0.259739,0.203549,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,0.0,0.0,0.0,0.0,0.011248,0.000946,1,64.0,different color,0,0.011248,0.000011,0.0
492,0.0,0.0,0.0,0.0,0.012067,0.000416,1,82.0,charge capacity,0,0.012067,0.000005,0.0
493,0.0,0.0,0.0,1.0,0.002920,0.000416,2,82.0,ticket number,0,0.005841,0.000002,0.0
494,0.0,0.0,0.0,0.0,0.046386,0.000000,1,182.0,recovery drive,0,0.046386,0.000000,0.0


Running 
drop table if exists pps_quality.EWS_MASTER_PS_CONSUMER;
 on aws3
Ran successfully!
Running 
alter table pps_quality.EWS_MASTER_PS_CONSUMER_PREV_1
                                rename to EWS_MASTER_PS_CONSUMER_PREV_2;
 on aws3
Failed!
Due to Severity: ROLLBACK, Message: A table named "EWS_MASTER_PS_CONSUMER_PREV_2" already exists, Sqlstate: 42710, Routine: validateRelationNameDoesNotExist, File: /scratch_a/release/svrtar18593/vbuild/vertica/Commands/DDL.cpp, Line: 20949, Error Code: 2105, SQL: 'alter table pps_quality.EWS_MASTER_PS_CONSUMER_PREV_1                                 rename to EWS_MASTER_PS_CONSUMER_PREV_2;'
Running 
alter table pps_quality.EWS_MASTER_PS_CONSUMER
                                rename to EWS_MASTER_PS_CONSUMER_PREV_1;
 on aws3
Failed!
Due to Severity: ROLLBACK, Message: Table "EWS_MASTER_PS_CONSUMER" does not exist, Sqlstate: 42704, Routine: RenameTable, File: /scratch_a/release/svrtar18593/vbuild/vertica/Commands/DDL.cpp, Line: 1800, Error Code:

...

0    444
1     31
Name: WoW, dtype: int64

Final record_count: 500


,WoW,CoCh,MAvg,HiLo,tfidf,ridf,total,count,terms,delta,we_sig_score,we_sig_ridf,we_sig_delta_ridf
0,0.0,1.0,1.0,0.0,0.710476,0.440672,3,13941.0,paper jam,0,2.131428,0.939261,0.0
1,0.0,0.0,1.0,1.0,0.416442,0.439456,3,2149.0,well regard,0,1.249327,0.549024,0.0
2,1.0,1.0,1.0,0.0,0.314186,0.414935,4,1267.0,onsite service,0,1.256742,0.521466,0.0
3,0.0,1.0,1.0,0.0,0.331029,0.301266,3,2859.0,paper jam issue,0,0.993088,0.299184,0.0
4,0.0,0.0,1.0,1.0,0.168321,0.519712,3,751.0,folder icon,0,0.504963,0.262435,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,0.0,0.0,0.0,0.0,0.013816,0.009712,1,122.0,print case,0,0.013816,0.000134,0.0
376,0.0,0.0,0.0,0.0,0.019338,0.006169,1,87.0,firmware bundle code,0,0.019338,0.000119,0.0
377,0.0,0.0,0.0,0.0,0.010828,0.009926,1,120.0,global print case,0,0.010828,0.000107,0.0
378,0.0,0.0,0.0,0.0,0.000000,0.274665,1,125.0,virtual agent,0,0.000000,0.000000,0.0


Running 
drop table if exists pps_quality.EWS_MASTER_PRINT_COMMERCIAL;
 on aws3
Ran successfully!
Running 
alter table pps_quality.EWS_MASTER_PRINT_COMMERCIAL_PREV_1
                                rename to EWS_MASTER_PRINT_COMMERCIAL_PREV_2;
 on aws3
Failed!
Due to Severity: ROLLBACK, Message: A table named "EWS_MASTER_PRINT_COMMERCIAL_PREV_2" already exists, Sqlstate: 42710, Routine: validateRelationNameDoesNotExist, File: /scratch_a/release/svrtar18593/vbuild/vertica/Commands/DDL.cpp, Line: 20949, Error Code: 2105, SQL: 'alter table pps_quality.EWS_MASTER_PRINT_COMMERCIAL_PREV_1                                 rename to EWS_MASTER_PRINT_COMMERCIAL_PREV_2;'
Running 
alter table pps_quality.EWS_MASTER_PRINT_COMMERCIAL
                                rename to EWS_MASTER_PRINT_COMMERCIAL_PREV_1;
 on aws3
Failed!
Due to Severity: ROLLBACK, Message: Table "EWS_MASTER_PRINT_COMMERCIAL" does not exist, Sqlstate: 42704, Routine: RenameTable, File: /scratch_a/release/svrtar18593/vbuild/vert

,WoW,CoCh,MAvg,HiLo,tfidf,ridf,total,count,terms,delta,we_sig_score,we_sig_ridf,we_sig_delta_ridf
0,0.0,1.0,1.0,0.0,0.710476,0.440672,3,13941.0,paper jam,0,2.131428,0.939261,0.0
1,0.0,0.0,1.0,1.0,0.416442,0.439456,3,2149.0,well regard,0,1.249327,0.549024,0.0
2,1.0,1.0,1.0,0.0,0.314186,0.414935,4,1267.0,onsite service,0,1.256742,0.521466,0.0
3,0.0,1.0,1.0,0.0,0.331029,0.301266,3,2859.0,paper jam issue,0,0.993088,0.299184,0.0
4,0.0,0.0,1.0,1.0,0.168321,0.519712,3,751.0,folder icon,0,0.504963,0.262435,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,0.0,0.0,0.0,0.0,0.013816,0.009712,1,122.0,print case,0,0.013816,0.000134,0.0
376,0.0,0.0,0.0,0.0,0.019338,0.006169,1,87.0,firmware bundle code,0,0.019338,0.000119,0.0
377,0.0,0.0,0.0,0.0,0.010828,0.009926,1,120.0,global print case,0,0.010828,0.000107,0.0
378,0.0,0.0,0.0,0.0,0.000000,0.274665,1,125.0,virtual agent,0,0.000000,0.000000,0.0


In [98]:
def populate_percent_contribution(tfidf,business,segment,iteration_suffix='current', 
                       threshold = 30, 
                       id_col = 'case_id', 
                       grouping_frequency = "W", 
                       verbose=False,date_col = 'case_closed_date',load_to_db = False):
    
    tables = get_tables(business=business, segment=segment, iteration_suffix=iteration_suffix)
    query = f"""select distinct lemmatized from pps_quality.{tables['ridf']} where ridf > 0;"""
    
    
    ridf, _ = hlp.get_data(query)
    ridf_terms = ridf.lemmatized.to_list()
    ridf_terms[:5]
    ridf_terms_str = "', '".join(ridf_terms)
    
    query = f"""select case_id, 
            date({date_col}) as case_closed_date,
            business, lemmatized
            from {hlp.schema}.{restructured_table}
            where date(case_closed_date) >= '{start_date}'
            and business = '{'Personal Systems' if business=='PS' else 'Printing'}'
            and lower(segment) = '{segment.lower()}'
            and lemmatized in (
                '{ridf_terms_str}'
            );"""

    with hlp.get_connection() as aws_conn:
        total_df = pd.read_sql(query, aws_conn,  parse_dates=['case_closed_date'])

    
    filtered = total_df.loc[total_df.lemmatized.map(total_df.lemmatized.value_counts()) >= threshold, :]
    
    if verbose:
        print(f'Total terms: {len(total_df)}')
        print(f'Total terms after applying threshold: {len(filtered)}')
        
    # Getting week wise counts
    
    grouped = filtered[[id_col, date_col, 'lemmatized']].groupby(['lemmatized', 
                                                                    pd.Grouper(key=date_col, 
                                                                               freq=grouping_frequency)
                                                                    ]).nunique().unstack()

    grouped.fillna(0, inplace=True)
    total_cases = len(filtered.case_id.unique())
    grouped = grouped.droplevel([0], axis=1)
    date_cols = [col for col in grouped.columns if col != 'lemmatized']
    
    temp = grouped.copy()
    col_rename_map = {}
    for col in date_cols:
        col_rename_map.update({col: col.strftime('%y-%W')})
        
    
    temp = temp.rename(col_rename_map, axis=1)
    temp.reset_index(inplace=True)    
    
    date_cols = list(col_rename_map.values())
    counts_df = pd.melt(temp, id_vars=['lemmatized'], value_vars=date_cols)
#     print(counts_df)
    
    temp = counts_df.drop(['lemmatized'],axis = 1)
    temp = temp.groupby(by=temp.case_closed_date).sum()
    temp.reset_index(inplace = True)
#     temp[['Year','Week']] = temp.case_closed_date.str.split("-",expand=True,)
#     dates = '20'+temp.Year+temp.Week
    
#     temp['case_closed_date'] = pd.to_datetime(dates.astype(str) + '0', format='%Y%W%w')
#     temp = temp.drop(['Year','Week'], axis=1)

    
    merged = tfidf.merge(temp, on = 'case_closed_date')
    merged['per_cont'] = merged['count']/merged['value']*100
    merged = merged.rename({'value':'total_count'}, axis=1)
#     display(merged)
    
    if load_to_db:
        selection_string = f'{business}_{segment}'
        hlp.load_to_vertica_database(merged,hlp.schema,f'EWS_PERCENT_{selection_string}')
    return merged

In [118]:
# businesses = [
#     'PS',
#               'PRINT'
# ]
# segments = [
#             'CONSUMER', 
#             'COMMERCIAL'
# ]

# for business in businesses:
#     for segment in segments:
#         tables = get_tables(business=business, 
#                         segment=segment, 
#                         iteration_suffix='current'
#                        )
#         tfidf, _ = hlp.get_data(f"""select lemmatized, case_closed_date, count, tfidf, tf from {hlp.schema}.{tables["tfidf"]}""")
#         tfidf.drop_duplicates(inplace=True)
#         print(tfidf)
#         df = populate_percent_contribution(tfidf,business,segment,load_to_db = True)


In [99]:
res.MAvg.value_counts()

0.0    287
1.0     93
Name: MAvg, dtype: int64

### Update Platform_name and product_category in restructured table

In [141]:
hlp.update_from_another_table('EWS_PYATE_NOTES_RESTRUCTURED', 'RDR_MASTER ',
                            tgt_key='case_id' , src_key= 'case_id' ,
                             tgt_cols = ['serial_number','product','end_user_company_name']
                             )

Running 
update pps_quality.EWS_PYATE_NOTES_RESTRUCTURED tgt
                set serial_number = src.serial_number, product = src.product, end_user_company_name = src.end_user_company_name
                from pps_quality.RDR_MASTER  src
                where tgt.case_id = src.case_id
            
 on aws3
Failed!
Due to Connection closed by Vertica


In [140]:
hlp.update_from_another_table('EWS_PYATE_NOTES_RESTRUCTURED', 'RDR_MASTER',
                             'case_id',
                             ['product_category', 'geo_market'],
                             update_only_nulls=True)

Running 
update pps_quality.EWS_PYATE_NOTES_RESTRUCTURED tgt
                set product_category = src.product_category, geo_market = src.geo_market
                from pps_quality.RDR_MASTER src
                where tgt.case_id = src.case_id
             and (tgt.product_category is null or tgt.geo_market is null)
 on aws3
Ran successfully!


In [53]:
hlp.update_from_another_table('EWS_PYATE_NOTES_RESTRUCTURED', 'RDR_MASTER ',
                            tgt_key='case_id' , src_key= 'case_id' ,
                             tgt_cols = ['Operating_System_Version']
                             )

Running 
update pps_quality.EWS_PYATE_NOTES_RESTRUCTURED tgt
                set Operating_System_Version = src.Operating_System_Version
                from pps_quality.RDR_MASTER  src
                where tgt.case_id = src.case_id
            
 on aws3
Failed!
Due to Severity: ERROR, Message: Column "Operating_System_Version" does not exist, Sqlstate: 42703, Routine: attnameAttNum, File: parse_relation.c, Line: 2059, Error Code: 2624, SQL: 'update pps_quality.EWS_PYATE_NOTES_RESTRUCTURED tgt                 set Operating_System_Version = src.Operating_System_Version                 from pps_quality.RDR_MASTER  src                 where tgt.case_id = src.case_id             '


### Add Parts Table Refresh

In [11]:
def get_missing_fields(table, attr_key='case_id'):
    
    missing_fields_query = f"""select distinct {attr_key}
                        from pps_quality.{table}
                        where 
                        {attr_key} not in (select distinct case_id from pps_quality.EWS_COST_AND_PARTS)
                        and business='Printing' and segment='Commercial'
                        and ip1 is not null 
                        and product not in ('INSTANTINK2')
                        and case_closed_date >= '2021-01-01';
                        """
    with hlp.get_connection() as aws_conn:
        missing_fields = pd.read_sql(missing_fields_query, aws_conn)

    return missing_fields

missing_fields = get_missing_fields('RDR_MASTER', attr_key='case_id')

In [12]:
missing_fields.shape

(1384564, 1)

In [13]:
chunksize = 10000
num_blocks = int(len(missing_fields) / chunksize) + 1
dfs = np.array_split(missing_fields, num_blocks)
print(f'Total blocks: {len(dfs)}')
# missing_fields_case_ids = hlp.divide_into_chunks(missing_fields,5000)

Total blocks: 139


In [14]:
with hlp.get_connection('csdw') as csdw_conn:
    for i, df in enumerate(dfs):
        print(f'\nProcessing block {i}')
        case_ids = df.case_id.astype(str).to_list()
        missing_fields_str = "', '".join(case_ids)
        
        query = f"""
            select Part_Number,
            ID as case_id,
            Part_Order_Detail_Description,
            Product,
            Serial_Number,
            Selected_Quantity,
            Failure_Code,
            Subassembly,
            Subsystem

            from dbo.V_Fact_Part_Data 
            where selected_quantity is not null
            and ID in (
                '{missing_fields_str}'
            )
            and Order_Number_Created_On_Date >= '2021-01-01'
            ;
            """
        parts_df = pd.read_sql(query, csdw_conn)
        parts_df.case_id = parts_df.case_id.astype(int)
        
        if len(parts_df) == 0:
            print('No parts information found for the given case ids')
            continue
        
        query = f"""
            select 
            [Support Event Identifier] as case_id,
            Fix_Type,
            [Interventions Category] as interventions_category,
            [Activity Cost US Dollar Amount] as activity_cost_US_Dollar_amount,
            Standard_Cost_DCCA_USD_Amount

            from dbo.V_Fact_Finance_Interventions_All 
            where [Support Event Identifier] in (
                '{missing_fields_str}'
            )
            ;
            """
        
        finance_data = pd.read_sql(query, csdw_conn)
        finance_data.case_id = finance_data.case_id.astype(int)
#         print('\tExtracted finance data')
         
        merged = pd.merge(finance_data, parts_df, how='right', on='case_id')
        merged_temp = merged.copy()
        merged_temp = merged_temp.drop_duplicates()
        print(f'deduplicated count: {len(merged_temp)}')
#         display(merged)
        
        hlp.load_to_vertica_database(merged_temp, hlp.schema, 'EWS_COST_AND_PARTS')
print('Successful!')

dict_keys(['csdw', 'sql_cache', 'sql_cache_surge', 'qtools', 'qtools_aws', 'product_info', 'smart_fleet', 'a360'])

Processing block 0
deduplicated count: 10
	Number of rows loaded: 10
	Number of rows rejected: 0


Total Rows loaded: 10
Total rejected rows: 0

Processing block 1
deduplicated count: 12
	Number of rows loaded: 12
	Number of rows rejected: 0


Total Rows loaded: 12
Total rejected rows: 0

Processing block 2
No parts information found for the given case ids

Processing block 3
deduplicated count: 1
	Number of rows loaded: 1
	Number of rows rejected: 0


Total Rows loaded: 1
Total rejected rows: 0

Processing block 4
deduplicated count: 4
	Number of rows loaded: 4
	Number of rows rejected: 0


Total Rows loaded: 4
Total rejected rows: 0

Processing block 5
deduplicated count: 4
	Number of rows loaded: 4
	Number of rows rejected: 0


Total Rows loaded: 4
Total rejected rows: 0

Processing block 6
deduplicated count: 11
	Number of rows loaded: 11
	Number of rows rejected: 0



deduplicated count: 109
	Number of rows loaded: 109
	Number of rows rejected: 0


Total Rows loaded: 109
Total rejected rows: 0

Processing block 60
deduplicated count: 43
	Number of rows loaded: 43
	Number of rows rejected: 0


Total Rows loaded: 43
Total rejected rows: 0

Processing block 61
deduplicated count: 65
	Number of rows loaded: 65
	Number of rows rejected: 0


Total Rows loaded: 65
Total rejected rows: 0

Processing block 62
deduplicated count: 145
	Number of rows loaded: 145
	Number of rows rejected: 0


Total Rows loaded: 145
Total rejected rows: 0

Processing block 63
deduplicated count: 17
	Number of rows loaded: 17
	Number of rows rejected: 0


Total Rows loaded: 17
Total rejected rows: 0

Processing block 64
deduplicated count: 161
	Number of rows loaded: 161
	Number of rows rejected: 0


Total Rows loaded: 161
Total rejected rows: 0

Processing block 65
deduplicated count: 118
	Number of rows loaded: 118
	Number of rows rejected: 0


Total Rows loaded: 118
Total reje

deduplicated count: 375
	Number of rows loaded: 375
	Number of rows rejected: 0


Total Rows loaded: 375
Total rejected rows: 0

Processing block 115
deduplicated count: 410
	Number of rows loaded: 410
	Number of rows rejected: 0


Total Rows loaded: 410
Total rejected rows: 0

Processing block 116
deduplicated count: 331
	Number of rows loaded: 331
	Number of rows rejected: 0


Total Rows loaded: 331
Total rejected rows: 0

Processing block 117
deduplicated count: 449
	Number of rows loaded: 449
	Number of rows rejected: 0


Total Rows loaded: 449
Total rejected rows: 0

Processing block 118
deduplicated count: 304
	Number of rows loaded: 304
	Number of rows rejected: 0


Total Rows loaded: 304
Total rejected rows: 0

Processing block 119
deduplicated count: 524
	Number of rows loaded: 524
	Number of rows rejected: 0


Total Rows loaded: 524
Total rejected rows: 0

Processing block 120
deduplicated count: 415
	Number of rows loaded: 415
	Number of rows rejected: 0


Total Rows loaded:

In [15]:
update_query = """UPDATE pps_quality.EWS_COST_AND_PARTS set Failure_Code =
REPLACE(SUBSTRING(Failure_Code,1,1500),'72 - Returned good','Unused')"""
hlp.run_query(update_query)

Running 
UPDATE pps_quality.EWS_COST_AND_PARTS set Failure_Code =
REPLACE(SUBSTRING(Failure_Code,1,1500),'72 - Returned good','Unused')
 on aws3
Ran successfully!


True

In [18]:
q="""ALTER TABLE pps_quality.EWS_COST_AND_PARTS 
ADD segment varchar(2000)"""

hlp.run_query(q)

Running 
ALTER TABLE pps_quality.EWS_COST_AND_PARTS 
ADD segment varchar(2000)
 on aws3
Ran successfully!


True

In [19]:
hlp.update_from_another_table('EWS_COST_AND_PARTS', 'RDR_MASTER',
                             'case_id',
                             ['business', 'segment'],
                             update_only_nulls=True)

Running 
update pps_quality.EWS_COST_AND_PARTS tgt
                set business = src.business, segment = src.segment
                from pps_quality.RDR_MASTER src
                where tgt.case_id = src.case_id
             and (tgt.business is null or tgt.segment is null)
 on aws3
Ran successfully!


In [1]:
from kats.consts import TimeSeriesData
from kats.models.prophet import ProphetModel, ProphetParams
from kats.detectors.outlier import OutlierDetector
from kats.consts import TimeSeriesData, TimeSeriesIterator
from kats.detectors.cusum_detection import CUSUMDetector,MultiCUSUMDetector

ModuleNotFoundError: No module named 'kats'

In [3]:
pip install kats==0.1.0

     |████████████████████████████████| 6.3 MB 982 kB/s eta 0:00:01
     |████████████████████████████████| 60 kB 73.0 MB/s eta 0:00:01
     |████████████████████████████████| 934 kB 55.7 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 70.2 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 56.3 MB/s eta 0:00:01
     |████████████████████████████████| 310 kB 61.0 MB/s eta 0:00:01
     |████████████████████████████████| 172 kB 57.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 39.2 MB/s eta 0:00:01
     |████████████████████████████████| 27.7 MB 58.2 MB/s eta 0:00:01
     |████████████████████████▎       | 50.9 MB 73.7 MB/s eta 0:00:01ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device

You should consider upgrading via the '/home/ubuntu/Notebooks/DEV_ENV/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### Dictionary Effort Automation

In [126]:
q = """select lemmatized, count(*)
        from pps_quality.EWS_PYATE_NOTES_PROCESSED_RESTRUCTURED
        where business = 'Printing'
        and lemmatized is not null
        group by lemmatized
        having count(*) >= 100
        order by count(*) desc;"""

print_terms_df = hlp.get_sample_data(q, rows=10000000)
# hlp.analyse_df(print_terms_df)

DatabaseError: Execution failed on sql 'select lemmatized, count(*)
        from pps_quality.EWS_PYATE_NOTES_PROCESSED_RESTRUCTURED
        where business = 'Printing'
        and lemmatized is not null
        group by lemmatized
        having count(*) >= 100
        order by count(*) desc limit 10000000;': Severity: ERROR, Message: Relation "pps_quality.EWS_PYATE_NOTES_PROCESSED_RESTRUCTURED" does not exist, Sqlstate: 42V01, Routine: throwRelationDoesNotExist, File: /scratch_a/release/svrtar18593/vbuild/vertica/Catalog/CatalogLookup.cpp, Line: 3665, Error Code: 4568, SQL: "select lemmatized, count(*)         from pps_quality.EWS_PYATE_NOTES_PROCESSED_RESTRUCTURED         where business = 'Printing'         and lemmatized is not null         group by lemmatized         having count(*) >= 100         order by count(*) desc limit 10000000;"

In [127]:
print_terms = print_terms_df.lemmatized.to_list()
print_terms_str = "', '".join(print_terms)
print_terms[:5]

NameError: name 'print_terms_df' is not defined

In [ ]:
q = f"""select distinct case_id
        from pps_quality.EWS_PYATE_NOTES_PROCESSED_RESTRUCTURED
        where lemmatized in (
        '{print_terms_str}'
        );"""

with hlp.get_connection('aws3') as conn:
    relevant_df = pd.read_sql(q, conn)
    
relevant_df.head()

In [ ]:
# hlp.analyse_df(relevant_df)
case_id_str = "', '".join(relevant_df.case_id.astype(str).to_list())

In [ ]:
q = f"""
select case_id,
        ip2,
        notes_history_cleaned_v3,
        case_closed_date,
        business
from pps_quality.EWS_PYATE_NOTES_PROCESSED
where case_id in (
'{case_id_str}'
);
"""

df = hlp.get_sample_data(q, rows=1000000000000)
hlp.analyse_df(df)

In [ ]:
from helper_pkg.pyate import PYATE
pyate = PYATE()

pyate.apply(relevant_df)

### Generate Network Graph - (Gephi)

### Rough Work

In [3]:
# from helper_pkg import Helper
# hlp = Helper()


In [1]:
# hlp.update_from_another_table('EWS_PYATE_NOTES_RESTRUCTURED', 'RDR_MASTER',
#                              'case_id',
#                              ['product_category', 'geo_market'])

In [2]:
# q = """update pps_quality.EWS_PYATE_NOTES_RESTRUCTURED tgt
#                 set product_category = src.product_category, geo_market = src.geo_market
#                 from pps_quality.RDR_MASTER src
#                 where tgt.case_id = src.case_id
#                 and date(src.case_closed_date) >= '2020-11-01'
#                 and (tgt.product_category is null or tgt.geo_market is null);
#                 """

# hlp.run_query(q)

In [153]:
# q = """select * from pps_quality.EWS_DICTIONARY_PS_UPDATED"""

# with hlp.get_connection() as aws_conn:
#     df = pd.read_sql(q, aws_conn)

In [155]:
# df.to_excel('output/missing_terms_ps_dictionary_2021_05_18.xlsx')

In [ ]:
q="""ALTER TABLE pps_quality.EWS_PYATE_RESTRUCTURED
        ADD Resolution_Status varchar(255);"""
hlp.run_query(q)

In [ ]:
hlp.update_from_another_table('RI_PYATE_RESTRUCTURED_v1', 'RDR_MASTER',
                             'case_id',
                             ['ip3'],
                             update_only_nulls=True)

In [4]:
query="""select * from pps_quality.EWS_PYATE_NOTES_PROCESSED
where date(case_closed_date)='2021-11-25'"""

with hlp.get_connection() as conn:
    tris = pd.read_sql(query, conn)
tris

,case_id,ip2,notes_history,case_closed_date,business,platform_name,country_code,notes_history_cleaned_v3,geo_market,term_1,...,term_18,score_18,count_18,term_19,score_19,count_19,term_20,score_20,count_20,segment
0,5059212870,services_and_repair,Log Type : Customer Log\r\nAction Type : Inbou...,2021-11-25T12:09:44.000Z,Personal Systems,Snape_20C1,US,order status for bpxx 01 . chat got disconnec...,None,order status,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Consumer
1,5060765539,sw_installation,Log Type : Customer Log\r\nAction Type : Inbou...,2021-11-25T12:17:29.000Z,Printing,MANHATTAN I-INK,US,problem description: wireless connection issue...,None,wireless connection issues,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Consumer
2,5062284616,services_and_repair,Log Type : Phone Log\r\nAction Type : Inbound ...,2021-11-25T15:56:02.000Z,Personal Systems,Lannister_19C2,US,eta feb 22nd . no arms raised yet . order plac...,None,order log,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Consumer
3,5065787932,sw_installation,Log Type : Phone Log\r\nAction Type : Inbound ...,2021-11-25T10:20:42.000Z,Printing,Skyreach,US,problem description: . issue:setup on desktop ...,None,desktop pc,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Consumer
4,5068257557,services_and_repair,Log Type : Notes Log \nAction Type : Inbou...,2021-11-25T12:16:14.000Z,Personal Systems,Ovenbird 1.0,US,note type . order reference number from servic...,None,order reference,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Commercial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18567,5076420894,cso_status_shipment,Log Type : 电话记录\r\nAction Type : Inbound Custo...,2021-11-25T22:48:02.000Z,Printing,POSEIDON WL,CN,关联,None,nan,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Consumer
18568,5076421033,os_load_issues,Log Type : 电话记录\r\nAction Type : Inbound Custo...,2021-11-25T23:21:13.000Z,Personal Systems,Atayal,CN,nps pass . end,None,nan,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Commercial
18569,5076421259,recovery_other,Log Type : Customer Log\r\nAction Type : Inbou...,2021-11-25T23:19:48.000Z,Personal Systems,Holmes_19C2,US,i need to create a recovery disc for my pc . ...,None,auto repair,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Consumer
18570,5076422271,hardware_third_party,Log Type : Phone Log\r\nAction Type : Inbound ...,2021-11-25T23:37:13.000Z,Personal Systems,Morpheus 2.2 PAV,US,"customer issue: hp assistant , printer update,...",None,printer app,...,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,Consumer


In [5]:
q = f"""drop table if exists pps_quality.'EWS_PYATE_PROCESSED_SPECIFIC';"""
hlp.run_query(q)

hlp.load_to_vertica_database(tris, hlp.schema, 'EWS_PYATE_PROCESSED_SPECIFIC')

Running 
drop table if exists pps_quality.'EWS_PYATE_PROCESSED_SPECIFIC';
 on aws3
Ran successfully!
Creating table since PPS_QUALITY.EWS_PYATE_PROCESSED_SPECIFIC not found

Create table query: 
create table PPS_QUALITY.EWS_PYATE_PROCESSED_SPECIFIC ( "case_id" integer, "ip2" varchar(1030), "notes_history" varchar(65000), "case_closed_date" varchar(1024), "business" varchar(1016), "platform_name" varchar(1053), "country_code" varchar(1004), "notes_history_cleaned_v3" varchar(29776), "geo_market" varchar(1004), "term_1" varchar(1043), "score_1" float, "count_1" float, "term_2" varchar(1043), "score_2" float, "count_2" float, "term_3" varchar(1047), "score_3" float, "count_3" float, "term_4" varchar(1043), "score_4" float, "count_4" float, "term_5" varchar(1054), "score_5" float, "count_5" float, "term_6" varchar(1044), "score_6" float, "count_6" float, "term_7" varchar(1038), "score_7" float, "count_7" float, "term_8" varchar(1052), "score_8" float, "count_8" float, "term_9" varchar(1055

'Successful'

In [6]:
id_col = 'case_id'
target_col = 'notes_history_cleaned_v3'
date_col = 'case_closed_date'

processed_table = f'EWS_PYATE_PROCESSED_SPECIFIC'
restructured_table = f'EWS_PYATE_NOTES_RESTRUCTURED_SPEC'

In [ ]:
def __init__(self) -> None:
        Helper.__init__(self)

        self.test = None
        # self.nlp = spacy.load('en_core_web_lg', disable=[
        #                       'ner', 'parser'])
        # self.nlp.add_pipe(TermExtractionPipeline())

        remove_dict = {
            r'^=': '',
            r'^-': '',
            r'--+': ' ',
            r'===+': ' ',
            r'\n+': ' ',
            r'(\>\>|\=\>)': '.',
            r'\t+': ' ',
            r' +': ' ',
            r'\S+@\S+': ' ',
            #                 r'\d\d\d\d\d+': '',
            r'\*+': '',
            r'#+': '',
            r'\s•\s': ' ',
            r'[_\'\"-\(\)\^|]+': '',
            r';+': '.',
            r'[<\[\]]+': '',
            r'[\{\}]+': '',
            r'- -': '',
            #                 r'\d\d:\d\d': '',
            r'\d:\d\d (a|p)m [a-z]*:': '',
            r'\d:\d\d (a|p)m': '',
            #                 r'\d\d:\d\d': ''
            '4th floor','8th floor','able print','above information',''
        }
        self.remove_patterns = {}
        for remove_ele, sub_with in remove_dict.items():
            pattern = re.compile(remove_ele)
            self.remove_patterns.update({pattern: sub_with})